# Import Thư Viện và Code Xử Lý

In [12]:
import imp
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pathlib import Path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from django.conf import settings
import re

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

from fuzzywuzzy import fuzz

# ****************************************Format**********************************************

from deep_translator import GoogleTranslator
from selenium.common.exceptions import NoSuchElementException
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}


# ****************************************Format**********************************************


def translator(text):
    """
    Dịch văn bản từ ngôn ngữ hiện tại sang tiếng Anh.
    # Đối với các trang quốc tế: Amazon, GadgetsNow, Reliance Digital
    """

    type_trans = GoogleTranslator(source='auto', target='en')
    trans = type_trans.translate(text)
    return trans


def no_accent_vietnamese(s):
    s = s.lower()
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('đ', 'd', s)
    return s


def convert(a):
    b = a.replace(" ", '')
    c = b.replace("INR", '')
    d = c.replace(",", '')
    d = d.replace("`", '')
    f = d.replace("₹", '')
    f1 = f.replace(".", '')
    f2 = f1.replace("₫", '')
    g = int(float(f2))
    return g


def format_price(price):
    """Formats a price number into a string with commas, currency symbol, and handles potential decimals.

  Args:
      price: The price as a number (float or int).

  Returns:
      A string representing the formatted price.
  """
    # Handle potential decimals
    if isinstance(price, float):
        price_str = "{:,.2f}".format(price)  # Two decimal places for floats
    else:
        price_str = "{:,}".format(price)  # No decimal places for integers

    return price_str + "đ"  # Add currency symbol (đ)
# **********************************************************************************************

# Crawl dữ liệu từ trang web

## Điện máy xanh

In [3]:


# ****************************************Điện Máy Xanh*****************************************
def dienmayxanh(name):
    try:
        global dienmayxanh
        # name=no_accent_vietnamese(name)
        name1 = name.replace(" ", "-")
        name2 = name.replace(" ", "+")

        dienmayxanh = f'https://www.dienmayxanh.com/tim-kiem?key={name2}'
        dienmayxanh_link = dienmayxanh

        res = requests.get(
            f'https://www.dienmayxanh.com/tim-kiem?key={name2}', headers=headers)
        print("\nSearching in Điện Máy Xanh...")
        soup = BeautifulSoup(res.text, 'html.parser')
        # print(soup)

        dienmayxanh_page = soup.select('a.main-contain')
        # print(dienmayxanh_page)

        dienmayxanh_page_length = int(len(dienmayxanh_page))
        # print(dienmayxanh_page_length)
        matching_products = []  # Danh sách sản phẩm tương đồng
        for i in range(0, dienmayxanh_page_length):
            name1 = name.upper()
            dienmayxanh_name = soup.select(
                'a.main-contain>h3')[i].getText().strip().upper()
            similarity_score = fuzz.ratio(name.upper(), dienmayxanh_name)
            if similarity_score > 50:
                dienmayxanh_name = soup.select(
                    'a.main-contain>h3')[i].getText().strip()

                dienmayxanh_images = soup.select('a.main-contain')

                # Kiểm tra xem có ảnh được trả về không
                if dienmayxanh_images:
                    # Nếu có ảnh, thực hiện lấy đường link ảnh
                    if dienmayxanh_images[i].find('img').get('src'):
                        dienmayxanh_image = dienmayxanh_images[i].find('img')[
                            'src']
                    elif dienmayxanh_images[0].find_all('img', class_='lazyload'):
                        dienmayxanh_image = dienmayxanh_images[0].find_all(
                            'img', class_='lazyload')[0]['data-src']
                    else:
                        # Nếu không có cả 'src' và 'data-src', gán một giá trị mặc định
                        dienmayxanh_image = '0'
                else:
                    # Nếu không có ảnh, gán một giá trị mặc định
                    dienmayxanh_image = '0'

                dienmayxanh_price = soup.select(
                    'a.main-contain>strong.price')[i].getText().strip().upper()
                dienmayxanh_price = dienmayxanh_price.strip('₫')
                print("Điện Máy Xanh:")
                print("Tên Sản Phẩm:", dienmayxanh_name)
                print("Giá:", dienmayxanh_price)
                print("Link Ảnh:", dienmayxanh_image)
                print("Link:", dienmayxanh_link)

                print("---------------------------------")
                break
            else:
                i += 1
                i = int(i)
                if i == dienmayxanh_page_length:
                    dienmayxanh_price = '0'
                    print("Điện Máy Xanh : No product found!")
                    print("-----------------------------")
                    break

        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_link
    except:
        print("Điện Máy Xanh: No product found!")
        print("---------------------------------")
        dienmayxanh_price = '0'
        dienmayxanh_name = '0'
        dienmayxanh_link = '0'
        dienmayxanh_image = '0'
    return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_link

# ******************************************************************************************************


# Test
dienmayxanh("Điện thoại Samsung Galaxy")


Searching in Điện Máy Xanh...
Điện Máy Xanh:
Tên Sản Phẩm: Samsung Galaxy S23 Ultra 5G
Giá: 21.990.000
Link Ảnh: https://cdn.tgdd.vn/Products/Images/42/249948/samsung-galaxy-s23-ultra-green-thumbnew-600x600.jpg
Link: https://www.dienmayxanh.com/tim-kiem?key=Điện+thoại+Samsung+Galaxy
---------------------------------


('21.990.000',
 'Samsung Galaxy S23 Ultra 5G',
 'https://cdn.tgdd.vn/Products/Images/42/249948/samsung-galaxy-s23-ultra-green-thumbnew-600x600.jpg',
 'https://www.dienmayxanh.com/tim-kiem?key=Điện+thoại+Samsung+Galaxy')

In [37]:

# NEW VERSION DONE

from bs4 import BeautifulSoup
import requests
from fuzzywuzzy import fuzz

# Assuming you have defined `headers` for requests


def dienmayxanh(name):
    try:
        name2 = name.replace(" ", "+")
        dienmayxanh_url = f'https://www.dienmayxanh.com/tim-kiem?key={name2}'

        res = requests.get(dienmayxanh_url, headers=headers)
        print("\nSearching in Điện Máy Xanh...")
        soup = BeautifulSoup(res.text, 'html.parser')

        dienmayxanh_page = soup.select('a.main-contain')
        dienmayxanh_page_length = int(len(dienmayxanh_page))
        # print(f"Number of products found: {dienmayxanh_page_length}")

        matching_products = []  # List to store matching products
        for i in range(0, dienmayxanh_page_length):
            dienmayxanh_name = soup.select(
                'a.main-contain>h3')[i].getText().strip().upper()

            similarity_score = fuzz.ratio(name.upper(), dienmayxanh_name)
            # print(f"Similarity Score: {similarity_score}")

            if similarity_score >= 10:
                dienmayxanh_name = soup.select(
                    'a.main-contain>h3')[i].getText().strip()
                # print(f"Name: {dienmayxanh_name}")

                dienmayxanh_images = soup.select('a.main-contain')
                if dienmayxanh_images:
                    if dienmayxanh_images[i].find('img').get('src'):
                        dienmayxanh_image = dienmayxanh_images[i].find('img')[
                            'src']
                    elif dienmayxanh_images[i].find_all('img', class_='lazyload'):
                        dienmayxanh_image = dienmayxanh_images[i].find_all(
                            'img', class_='lazyload')[0]['data-src']
                    elif dienmayxanh_images[i].find_all('img', class_='lazyloaded'):
                        dienmayxanh_image = dienmayxanh_images[i].find_all(
                            'img', class_='lazyloaded')[0]['data-src']
                    else:
                        dienmayxanh_image = '0'
                else:
                    dienmayxanh_image = '0'
                # print(f"Image: {dienmayxanh_image}")

                try:
                    # Robust empty price check
                    if not soup.select('a.main-contain>strong.price')[i].getText().strip() or soup.select('a.main-contain>strong.price')[i].getText().strip() == ' ':
                        dienmayxanh_price = '0'
                    else:
                        dienmayxanh_price = soup.select(
                            'a.main-contain>strong.price')[i].getText().strip().upper()
                        dienmayxanh_price = dienmayxanh_price.strip('₫')
                        # <strong class="price">Dự kiến: 44.990.000₫</strong>
                        dienmayxanh_price = re.sub(
                            "[^0-9]", "", dienmayxanh_price)
                except:
                    # print(f"Product {dienmayxanh_name} at index {i} has no price. Skipping.")
                    continue

                product = {
                    "name": dienmayxanh_name,
                    "price": dienmayxanh_price,
                    "image": dienmayxanh_image,
                    "link": dienmayxanh_url
                }

                # Kiểm tra xem sản phẩm nào price=0 thì không lưu vào matching_products và xoá product đó
                if product["price"] == '0':
                    del product
                else:
                    matching_products.append(product)

        # Nếu không có sản phẩm nào thì trả về None
        if not matching_products:
            return None

        # ------------------------------------------------------------------------
        # print("Matching Products:")
        # for product in matching_products:
        #     print("Tên Sản Phẩm:", product["name"])
        #     print("Giá:", product["price"])
        #     print("Link Ảnh:", product["image"])
        #     print("Link:", product["link"])
        #     print("---------------------------------")

        # # Find the cheapest product
        # cheapest_product = min(matching_products, key=lambda p: int(p["price"].replace('.', '')))

        # if cheapest_product:
        #     print("Tên Sản Phẩm:", cheapest_product["name"])
        #     print("Giá:", cheapest_product["price"])
        #     print("Link Ảnh:", cheapest_product["image"])
        #     print("Link:", cheapest_product["link"])
        #     print("---------------------------------")
        # else:
        #     print("No matching product found.")

        # Lưu cheapest_product vào list_product
        # list_product = []
        # list_product.append(cheapest_product)
        # ------------------------------------------------------------------------

        dienmaycholon_price = matching_products[0]["price"]
        dienmaycholon_name = matching_products[0]["name"]
        dienmaycholon_image = matching_products[0]["image"]
        dienmaycholon_url = matching_products[0]["link"]
        print("Điện Máy Xanh:")
        print("Tên Sản Phẩm:", dienmaycholon_name)
        print("Giá:", dienmaycholon_price)
        print("Link Ảnh:", dienmaycholon_image)
        print("Link:", dienmaycholon_url)
        print("---------------------------------")

        return dienmaycholon_price, dienmaycholon_name[0:50], dienmaycholon_image, dienmaycholon_url

    except Exception as e:
        print(f"Lỗi: {e}")
        print("Điện Máy Xanh: No product found!")
        print("---------------------------------")
        return "0", "0", "0", "0"


# Example usage
name = "macbook pro 14 inch"
product = dienmayxanh(name)


Searching in Điện Máy Xanh...
Điện Máy Xanh:
Tên Sản Phẩm: MacBook Pro 14 inch M3
Giá: 44990000
Link Ảnh: https://cdn.tgdd.vn/Products/Images/44/320981/macbook-pro-14-inch-m3-2023-16gb-thumb-600x600.jpg
Link: https://www.dienmayxanh.com/tim-kiem?key=macbook+pro+14+inch
---------------------------------


In [48]:
anh = "Hoài Thương"
be = "Thị Hoài Thương"
similarity_score = fuzz.ratio(anh, be)
similarity_score

85

## Thế giới di động

In [7]:
# ****************************************Thế giới di động**********************************************
def tgdd(name):
    try:
        global tgdd
        # name=no_accent_vietnamese(name)
        name1 = name.replace(" ", "-")
        name2 = name.replace(" ", "+")
        tgdd = f'https://www.thegioididong.com/tim-kiem?key={name2}'
        tgdd_link = tgdd
        res = requests.get(
            f'https://www.thegioididong.com/tim-kiem?key={name2}', headers=headers)
        print("\nSearching in Thế giới di động...")
        soup = BeautifulSoup(res.text, 'html.parser')
        # print(soup)
        tgdd_page = soup.select('a.main-contain')
        # print(tgdd_page)
        tgdd_page_length = int(len(tgdd_page))
        print(tgdd_page_length)
        for i in range(0, tgdd_page_length):
            name1 = name.upper()
            tgdd_name = soup.select(
                'a.main-contain>h3')[i].getText().strip().upper()
            similarity_score = fuzz.ratio(name.upper(), tgdd_name)
            if similarity_score > 5:
                tgdd_name = soup.select(
                    'a.main-contain>h3')[i].getText().strip()

                tgdd_images = soup.select('a.main-contain')

                # Kiểm tra xem có ảnh được trả về không
                if tgdd_images:
                    # Nếu có ảnh, thực hiện lấy đường link ảnh
                    if tgdd_images[i].find('img').get('src'):
                        tgdd_image = tgdd_images[i].find('img')['src']
                    elif tgdd_images[0].find_all('img', class_='lazyload'):
                        tgdd_image = tgdd_images[0].find_all(
                            'img', class_='lazyload')[0]['data-src']
                    else:
                        # Nếu không có cả 'src' và 'data-src', gán một giá trị mặc định
                        tgdd_image = '0'
                else:
                    # Nếu không có ảnh, gán một giá trị mặc định
                    tgdd_image = '0'

                tgdd_price = soup.select(
                    'a.main-contain>strong.price')[i].getText().strip().upper()
                tgdd_price = tgdd_price.strip('₫')
                print("Thế giới di động:")
                print("Tên Sản Phẩm:", tgdd_name)
                print("Giá:", tgdd_price)
                print("Link Ảnh:", tgdd_image)
                print("Link:", tgdd_link)
                print("---------------------------------")
                break
            else:
                i += 1
                i = int(i)
                if i == tgdd_page_length:
                    tgdd_price = '0'
                    print("Thế giới di động : No product found!")
                    print("-----------------------------")
                    break

        return tgdd_price, tgdd_name[0:50], tgdd_image, tgdd_link
    except:
        print("Thế giới di động: No product found!1")
        print("---------------------------------")
        tgdd_price = '0'
        tgdd_name = '0'
        tgdd_link = '0'
        tgdd_image = '0'
    return tgdd_price, tgdd_name[0:50], tgdd_image, tgdd_link
# **********************************************************************************************


# Test
tgdd("Điện thoại")


Searching in Thế giới di động...
20
Thế giới di động:
Tên Sản Phẩm: iPhone 15 Pro Max
Giá: 29.590.000
Link Ảnh: https://cdn.tgdd.vn/Products/Images/42/305658/iphone-15-pro-max-blue-thumbnew-600x600.jpg
Link: https://www.thegioididong.com/tim-kiem?key=Điện+thoại
---------------------------------


('29.590.000',
 'iPhone 15 Pro Max',
 'https://cdn.tgdd.vn/Products/Images/42/305658/iphone-15-pro-max-blue-thumbnew-600x600.jpg',
 'https://www.thegioididong.com/tim-kiem?key=Điện+thoại')

## Amazon

In [9]:
# # ****************************************Amazon**********************************************
def amazon(name):
    try:
        global amazon
        name = translator(name)
        name1 = name.replace(" ", "-")
        name2 = name.replace(" ", "+")
        amazon = f'https://www.amazon.in/{name1}/s?k={name2}'
        amazon_link = amazon
        res = requests.get(
            f'https://www.amazon.in/{name1}/s?k={name2}', headers=headers)
        print("\nSearching in amazon...")
        soup = BeautifulSoup(res.text, 'html.parser')
        amazon_page = soup.select('.a-color-base.a-text-normal')
        amazon_page_length = int(len(amazon_page))
        for i in range(0, amazon_page_length):
            name = name.upper()
            amazon_name = soup.select(
                '.a-color-base.a-text-normal')[i].getText().strip().upper()

            similarity_score = fuzz.ratio(name.upper(), amazon_name)

            # if name in amazon_name:
            if similarity_score > 5:
                amazon_name = soup.select(
                    '.a-color-base.a-text-normal')[i].getText().strip()
                amazon_images = soup.select(
                    '.a-section.aok-relative.s-image-fixed-height')
                amazon_image = amazon_images[0].find_all(
                    'img', class_='s-image')[0]
                amazon_image = amazon_image['src']
                amazon_price = soup.select(
                    '.a-price-whole')[i].getText().strip().upper()
                print("Amazon:")
                print("Tên Sản Phẩm:", amazon_name)
                print("Giá:", amazon_price)
                print("Link Ảnh:", amazon_image)
                print("Link:", amazon_link)
                print("---------------------------------")
                break
            else:
                i += 1
                i = int(i)
                if i == amazon_page_length:
                    amazon_price = '0'
                    print("amazon : No product found!")
                    print("-----------------------------")
                    break

        return amazon_price, amazon_name[0:50], amazon_image, amazon_link
    except:
        print("Amazon: No product found!")
        print("---------------------------------")
        amazon_price = '0'
        amazon_name = '0'
        amazon_link = '0'
        amazon_image = '0'
    return amazon_price, amazon_name[0:50], amazon_image, amazon_link


# **********************************************************************************************
# Test
amazon("Điện thoại")


Searching in amazon...
Amazon:
Tên Sản Phẩm: Apple iPhone 13 (128GB) - Midnight
Giá: 48,499
Link Ảnh: https://m.media-amazon.com/images/I/61VuVU94RnL._AC_UY218_.jpg
Link: https://www.amazon.in/Phone/s?k=Phone
---------------------------------


('48,499',
 'Apple iPhone 13 (128GB) - Midnight',
 'https://m.media-amazon.com/images/I/61VuVU94RnL._AC_UY218_.jpg',
 'https://www.amazon.in/Phone/s?k=Phone')

In [16]:
#  ****************************************Amazon**********************************************
def amazon(name):
    try:
        global amazon
        name = translator(name)
        name1 = name.replace(" ", "-")
        name2 = name.replace(" ", "+")
        amazon = f'https://www.amazon.in/{name1}/s?k={name2}'
        amazon_link = amazon
        print(amazon_link)
        res = requests.get(
            f'https://www.amazon.in/{name1}/s?k={name2}', headers=headers)
        print("\nSearching in amazon...")
        soup = BeautifulSoup(res.text, 'html.parser')
        amazon_page = soup.select('.a-color-base.a-text-normal')
        amazon_page_length = int(len(amazon_page))
        for i in range(0, amazon_page_length):
            name = name.upper()
            amazon_name = soup.select(
                '.a-color-base.a-text-normal')[i].getText().strip().upper()
            
            # Tính toán độ tương đồng giữa tên sản phẩm và tên sản phẩm tìm kiếm
            similarity_score = fuzz.ratio(name, amazon_name)
            # print(f"Similarity Score: {similarity_score}")
            if similarity_score >= 10:
                amazon_name = soup.select(
                    '.a-color-base.a-text-normal')[i].getText().strip()
                amazon_images = soup.select(
                    '.a-section.aok-relative.s-image-fixed-height')
                amazon_image = amazon_images[0].find_all(
                    'img', class_='s-image')[0]
                amazon_image = amazon_image['src']
                amazon_price = soup.select(
                    '.a-price-whole')[i].getText().strip().upper()
                print("Amazon:")
                print("Tên Sản Phẩm:", amazon_name)
                print("Giá:", amazon_price)
                print("Link Ảnh:", amazon_image)
                print("Link:", amazon_link)
                print("---------------------------------")
                break
            else:
                i += 1
                i = int(i)
                if i == amazon_page_length:
                    amazon_price = '0'
                    print("amazon : No product found!")
                    print("-----------------------------")
                    break

        return amazon_price, amazon_name[0:50], amazon_image, amazon_link
    except:
        print("Amazon: No product found!")
        print("---------------------------------")
        amazon_price = '0'
        amazon_name = '0'
        amazon_link = '0'
        amazon_image = '0'
    return amazon_price, amazon_name[0:50], amazon_image, amazon_link
# ====================================================================================


amazon("Iphone 13 pro max")

https://www.amazon.in/Iphone-13-pro-max/s?k=Iphone+13+pro+max

Searching in amazon...
Amazon:
Tên Sản Phẩm: Apple iPhone 13 (256GB) - Pink
Giá: 58,999
Link Ảnh: https://m.media-amazon.com/images/I/61l9ppRIiqL._AC_UY218_.jpg
Link: https://www.amazon.in/Iphone-13-pro-max/s?k=Iphone+13+pro+max
---------------------------------


('58,999',
 'Apple iPhone 13 (256GB) - Pink',
 'https://m.media-amazon.com/images/I/61l9ppRIiqL._AC_UY218_.jpg',
 'https://www.amazon.in/Iphone-13-pro-max/s?k=Iphone+13+pro+max')

## Chợ tốt

In [49]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import ChromiumOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.common.by import By


def chotot(name):
    try:
        global chotot
        name1 = name.replace(" ", "-")
        name2 = name.replace(" ", "+")
        chotot = f'https://www.chotot.com/mua-ban?q={name2}'
        chotot_link = chotot

        service = Service(ChromeDriverManager().install())
        chrome_options = ChromiumOptions()
        chrome_options.add_argument("--headless")
        driver = webdriver.Chrome(options=chrome_options, service=service)
        driver.get(chotot)

        try:
            elementname = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "h3.commonStyle_adTitle__g520j"))
            )
            elementprice = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "p.AdBody_adPriceNormal___OYFU"))
            )
            imgelement = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "picture.webpimg-container img"))
            )
        except:
            driver.quit()

        chotot_name = elementname.text
        chotot_price = elementprice.text
        chotot_price = chotot_price.strip('đ')
        chotot_image = imgelement.get_attribute("src")
        print("Chợ tốt:")
        print("Tên Sản Phẩm:", chotot_name)
        print("Giá:", chotot_price)
        print("Link Ảnh:", chotot_image)
        print("Link:", chotot_link)
        print("---------------------------------")

        return chotot_price, chotot_name[0:50], chotot_image, chotot_link
    except:
        print("chotot: No product found!")
        print("---------------------------------")
        chotot_price = '0'
        chotot_name = '0'
        chotot_image = '0'
        chotot_link = '0'
    return chotot_price, chotot_name[0:50], chotot_image, chotot_link


name = "Macbook pro"
chotot_price, chotot_name, chotot_image, chotot_link = chotot(name)

Chợ tốt:
Tên Sản Phẩm: Macbook Pro 14 inch 2021 M1 Pro ram 32gb cực mạnh
Giá: 29.890.000 
Link Ảnh: https://cdn.chotot.com/NPItJWkVH3OG4ccU5ZtdqcJvzKMnyFHtlbdldbiT5Bk/preset:listing/plain/69d15cb082ef09198e71a8fa82b252bd-2876675725635260075.jpg
Link: https://www.chotot.com/mua-ban?q=Macbook+pro
---------------------------------


In [47]:
type(products)

list

## Sen đỏ

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import ChromiumOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.common.by import By

name = "Macbook pro"
name2 = name.replace(" ", "+")
sendo = f"https://www.sendo.vn/tim-kiem?q={name2}"
sendo_link = sendo

service = Service(ChromeDriverManager().install())
chrome_options = ChromiumOptions()
# chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options, service=service)
driver.get(sendo)

# Tìm kiếm danh sách element tên sản phẩm
name_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, "span.d7ed-Vp2Ugh._0032-Zwkt7j"))
)

# Lấy danh sách tên sản phẩm
sendo_names = []
for name_element in name_elements:
    # Lọc text để lấy tên sản phẩm
    sendo_name = name_element.text
    sendo_names.append(sendo_name)


# Tìm kiếm danh sách element giá sản phẩm
price_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, "span._0032-GpBMYp._0032-npoTU_.d7ed-CLUDGW.d7ed-AHa8cD.d7ed-giDKVr"))
)

# Lấy danh sách giá sản phẩm
sendo_prices = []
for price_element in price_elements:
    # Lấy text và xóa ký tự "đ"
    sendo_price = price_element.text.strip(
        'đ')  # Xóa dấu chấm phân cách hàng nghìn
    sendo_prices.append(sendo_price)


# Tìm kiếm danh sách element ảnh sản phẩm
image_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, "div.d7ed-a1ulZz img"))
)

# Lấy danh sách link ảnh sản phẩm
sendo_images = []
for image_element in image_elements:
    # Lấy thuộc tính src hoặc data-src
    image_src = image_element.get_attribute("src")
    if not image_src:  # Nếu src trống, lấy data-src
        image_src = image_element.get_attribute("data-src")
    sendo_images.append(image_src)

In [19]:
# Tìm kiếm danh sách element tên sản phẩm
name_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, "span.d7ed-Vp2Ugh._0032-Zwkt7j"))
)

# Lấy danh sách tên sản phẩm
sendo_names = []
for name_element in name_elements:
    # Lọc text để lấy tên sản phẩm
    sendo_name = name_element.text
    sendo_names.append(sendo_name)

# In danh sách tên sản phẩm
print(sendo_names)

['MacBook Pro 14 2023 M3 8 CPU/10 GPU/8GB/512GB - Xám', 'MacBook Pro 14 2023 M3 8 CPU/10 GPU/8GB/512GB - Bạc', 'MacBook Air 15 inch M2 2023 8CPU 10GPU 8GB/256GB - Xanh đen', 'MacBook Air 13 inch M2 2022 8CPU 8GPU 16GB/256GB - Xám', 'MacBook Air 15 inch M2 2023 8CPU 10GPU 8GB/256GB - Vàng', 'MacBook Air 15 inch M2 2023 8CPU 10GPU 8GB/256GB - Bạc', 'MacBook Air 15 inch M2 2023 8CPU 10GPU 8GB/256GB - Xám', 'MacBook Air 15 inch M2 2023 8CPU 10GPU 8GB/512GB - Xanh đen', 'MacBook Air 13 inch M2 2022 8CPU 10GPU 8GB/512GB - Vàng', 'MacBook Air 13 inch M2 2022 8CPU 10GPU 8GB/512GB - Bạc', 'MacBook Pro 14 2023 M3 Pro 11 CPU/14GPU/36GB/512GB - Đen', 'MacBook Pro 14 2023 M3 Pro 11 CPU/14 GPU/18GB/512GB - Đen', 'MacBook Pro 14 2023 M3 Pro 12 CPU/18 GPU/18GB/1TB - Bạc', 'MacBook Pro 14 2023 M3 Pro 12 CPU/18 GPU/18GB/1TB - Đen', 'MacBook Pro 14 2023 M3 Pro 11 CPU/14 GPU/18GB/512GB - Bạc', 'MacBook Pro Touchbar TUCH ID 13IN 2560 1600 Gray i5 8G 128G CHÍNH HÃNG', 'Macbook Pro 13.3', 'Macbook pro 13INCH

In [20]:
# ... (phần code trước đó) ...

# Tìm kiếm danh sách element giá sản phẩm
price_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, "span._0032-GpBMYp._0032-npoTU_.d7ed-CLUDGW.d7ed-AHa8cD.d7ed-giDKVr"))
)

# Lấy danh sách giá sản phẩm
sendo_prices = []
for price_element in price_elements:
    # Lấy text và xóa ký tự "đ"
    sendo_price = price_element.text.strip(
        'đ')  # Xóa dấu chấm phân cách hàng nghìn
    sendo_prices.append(sendo_price)

# In danh sách giá sản phẩm
print(sendo_prices)

['38.790.000', '38.790.000', '27.690.000', '34.990.000', '27.690.000', '27.990.000', '27.990.000', '32.990.000', '29.990.000', '29.990.000', '59.990.000', '49.490.000', '59.490.000', '59.490.000', '49.490.000', '17.999.000', '23.500.000', '26.999.000', '24.000.000', '23.999.000', '21.999.000', '39.999.000', '25.498.000', '19.989.000', '28.500.000', '23.000.000', '33.000.000', '45.500.000', '14.000.000', '20.500.000', '17.500.000', '33.550.000', '24.999.000', '7.000.000', '32.188.000', '56.100.000', '13.500.000', '19.496.000', '31.700.000', '35.000.000', '45.388.000', '27.120.000', '33.000.000', '17.500.000', '20.500.000', '22.500.000', '12.500.000', '11.000.000', '18.500.000', '22.500.000', '27.499.000', '35.796.000', '34.900.000', '19.500.000', '20.499.000', '16.989.000', '52.000.000', '7.000.000', '30.499.000', '41.996.000']


In [21]:
# Tìm kiếm danh sách element ảnh sản phẩm
image_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, "div.d7ed-a1ulZz img"))
)

# Lấy danh sách link ảnh sản phẩm
sendo_images = []
for image_element in image_elements:
    # Lấy thuộc tính src hoặc data-src
    image_src = image_element.get_attribute("src")
    if not image_src:  # Nếu src trống, lấy data-src
        image_src = image_element.get_attribute("data-src")
    sendo_images.append(image_src)

# In danh sách link ảnh sản phẩm
print(sendo_images)

['https://media3.scdn.vn/img4/2023/12_05/Occ4HWYUmSXX12ucsWBf_simg_de2fe0_250x250_maxb.jpg', 'https://media3.scdn.vn/img4/2023/12_05/oPrBSEmSsxEjwmQ4cpUs_simg_de2fe0_250x250_maxb.jpg', 'https://media3.scdn.vn/img4/2023/08_02/VF2UYtLp16vfp4C2weyb_simg_de2fe0_250x250_maxb.jpg', 'https://media3.scdn.vn/img4/2022/09_10/cGh0j4cKr72oJzfgwlB2_simg_de2fe0_250x250_maxb.jpg', 'https://media3.scdn.vn/img4/2023/08_02/bF7IkC9nQAaFIkNeg0nI_simg_de2fe0_250x250_maxb.jpg', 'https://media3.scdn.vn/img4/2023/08_02/v4cSMB2I7gp8D2RqgT10_simg_de2fe0_250x250_maxb.jpg', 'https://media3.scdn.vn/img4/2023/08_02/rcYJdPWClA8Ye4Prj0vu_simg_de2fe0_250x250_maxb.jpg', 'https://media3.scdn.vn/img4/2023/08_02/CODks1LfmjcVWTFz1c9m_simg_de2fe0_250x250_maxb.jpg', 'https://media3.scdn.vn/img4/2022/10_04/aJAnoxfbK374eSSQjgZ3_simg_de2fe0_250x250_maxb.jpg', 'https://media3.scdn.vn/img4/2022/10_04/J1BRWDXRaNG0F4Gmys3V_simg_de2fe0_250x250_maxb.jpg', 'https://media3.scdn.vn/img4/2024/02_16/74bNe844Yal7pjGgge6Z_simg_de2fe0_250x25

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import ChromiumOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.common.by import By


def sendo(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "+")
        sendo_url = f"https://www.sendo.vn/tim-kiem?q={name2}"

        # Khởi tạo driver (headless Chrome)
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(options=chrome_options, service=service)
        driver.get(sendo_url)

        # Tìm kiếm danh sách element
        name_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "span.d7ed-Vp2Ugh._0032-Zwkt7j"))
        )
        # Tìm kiếm danh sách element giá sản phẩm
        price_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "span._0032-GpBMYp._0032-npoTU_.d7ed-CLUDGW.d7ed-AHa8cD.d7ed-giDKVr"))
        )
        # Tìm kiếm danh sách element ảnh sản phẩm
        image_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "div.d7ed-a1ulZz img"))
        )

        # # Kiểm tra số lượng element
        num_elements = len(name_elements)
        # print(num_elements)
        if num_elements != len(price_elements) or num_elements != len(image_elements):
            print("sendo: Số lượng element không khớp!")
            driver.quit()
            return []

        # Lấy danh sách sản phẩm
        products = []
        min_price = float('inf')  # Khởi tạo giá trị min_price là vô cùng
        cheapest_product = None
        for i in range(num_elements):
            # Lấy tên sản phẩm
            sendo_name = name_elements[i].text

            # Lấy giá sản phẩm
            sendo_price = price_elements[i].text.strip('đ')

            # Chuyển đổi giá thành số (xóa dấu chấm)
            try:
                price_num = int(sendo_price.replace('.', ''))
            except ValueError:
                print(f"Lỗi chuyển đổi giá: {sendo_price}")
                continue  # Bỏ qua sản phẩm này nếu không thể chuyển đổi giá

            # Lấy link ảnh sản phẩm
            image_src = image_elements[i].get_attribute("data-src")
            if not image_src:
                image_src = image_elements[i].get_attribute("src")

            # Lưu thông tin vào dictionary
            product = {
                "name": sendo_name,
                "price": sendo_price,
                "image": image_src,
                "link": sendo_url
            }

            # Kiểm tra giá và cập nhật sản phẩm rẻ nhất
            if price_num < min_price:
                min_price = price_num
                cheapest_product = product

        # Thêm sản phẩm rẻ nhất vào danh sách (nếu có)
        if cheapest_product:
            products.append(cheapest_product)

        driver.quit()  # Đóng trình duyệt

        # In thông tin sản phẩm (nếu có)
        if products:
            for product in products:
                print("Sendo:")
                print("Tên Sản Phẩm:", product["name"])
                print("Giá:", product["price"])
                print("Link Ảnh:", product["image"])
                print("Link:", product["link"])
                print("---------------------------------")
        else:
            print("sendo: No product found!")
            print("---------------------------------")

        return products

    except Exception as e:
        print(f"Lỗi: {e}")
        return []


name = "iphone 13 pro max"
products = sendo(name)

Sendo:
Tên Sản Phẩm: [Clover 2020] Ốp lưng iPhone 13 Pro Max dẻo có màn bảo vệ camera
Giá: 12.000
Link Ảnh: https://media3.scdn.vn/img4/2021/09_12/QFq3lxPAz9woOTp4DVIu_simg_de2fe0_250x250_maxb.jpg
Link: https://www.sendo.vn/tim-kiem?q=iphone+13+pro+max
---------------------------------


In [20]:
products

[{'name': 'Macbook Pro A1286 15in, i5 4G SSD128 Vga rời Đẹp zin 100 Giá rẻ',
  'price': '7.000.000',
  'image': 'https://media3.scdn.vn/img4/2020/02_20/blQOXTGXHlzNOuV1j9sX_simg_de2fe0_250x250_maxb.jpg',
  'link': 'https://www.sendo.vn/tim-kiem?q=Macbook+pro'}]

In [56]:
def sendo(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "+")
        sendo_url = f"https://www.sendo.vn/tim-kiem?q={name2}"

        # Khởi tạo driver (headless Chrome)
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(options=chrome_options, service=service)
        driver.get(sendo_url)

        # Tìm kiếm danh sách element
        name_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "span.d7ed-Vp2Ugh._0032-Zwkt7j"))
        )
        # Tìm kiếm danh sách element giá sản phẩm
        price_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "span._0032-GpBMYp._0032-npoTU_.d7ed-CLUDGW.d7ed-AHa8cD.d7ed-giDKVr"))
        )
        # Tìm kiếm danh sách element ảnh sản phẩm
        image_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "div.d7ed-a1ulZz img"))
        )

        # # Kiểm tra số lượng element
        num_elements = len(name_elements)
        # print(num_elements)
        if num_elements != len(price_elements) or num_elements != len(image_elements):
            print("sendo: Số lượng element không khớp!")
            driver.quit()
            return []

        # Lấy danh sách sản phẩm
        products = []

        for i in range(num_elements):
            similarity_score = fuzz.ratio(
                name.upper(), name_elements[i].text.upper())
            if similarity_score >= 10:
                # Lấy tên sản phẩm
                sendo_name = name_elements[i].text

                # Lấy giá sản phẩm
                sendo_price = price_elements[i].text.strip('đ')

                # Lấy link ảnh sản phẩm
                image_src = image_elements[i].get_attribute("data-src")
                if not image_src:
                    image_src = image_elements[i].get_attribute("src")

                # Lưu thông tin vào dictionary
                product = {
                    "name": sendo_name,
                    "price": sendo_price,
                    "image": image_src,
                    "link": sendo_url
                }

                products.append(product)

        driver.quit()  # Đóng trình duyệt

        # Lấy sản phẩm đầu
        sendo_name = products[0]["name"]
        sendo_price = products[0]["price"]
        sendo_image = products[0]["image"]
        sendo_link = products[0]["link"]

        if products:
            print("Sendo:")
            print("Tên Sản Phẩm:", sendo_name)
            print("Giá:", sendo_price)
            print("Link Ảnh:", sendo_image)
            print("Link:", sendo_link)
            print("---------------------------------")
            return sendo_price, sendo_name[0:50], sendo_image, sendo_link

        else:
            print("sendo: No product found!")
            print("---------------------------------")
            sendo_price = '0'
            sendo_name = '0'
            sendo_link = '0'
            sendo_image = '0'
            return sendo_price, sendo_name, sendo_image, sendo_link

    except Exception as e:
        print(f"Lỗi: {e}")
        sendo_price = '0'
        sendo_name = '0'
        sendo_link = '0'
        sendo_image = '0'
        return sendo_price, sendo_name, sendo_image, sendo_link


name = "Iphone 15 pro"
products = sendo(name)

Sendo:
Tên Sản Phẩm: iPhone 15 Pro Max 256GB - Titan Tự nhiên
Giá: 29.990.000
Link Ảnh: https://media3.scdn.vn/img4/2023/09_27/t9avGPZxfnwqaJvn1hI2_simg_de2fe0_250x250_maxb.jpg
Link: https://www.sendo.vn/tim-kiem?q=Iphone+15+pro
---------------------------------


In [57]:
products

('29.990.000',
 'iPhone 15 Pro Max 256GB - Titan Tự nhiên',
 'https://media3.scdn.vn/img4/2023/09_27/t9avGPZxfnwqaJvn1hI2_simg_de2fe0_250x250_maxb.jpg',
 'https://www.sendo.vn/tim-kiem?q=Iphone+15+pro')

---

## ĐIỆN MÁY CHỢ LỚN

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import ChromiumOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.common.by import By


def dienmaycholon(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "+")
        dienmaycholon_url = f"https://dienmaycholon.vn/tu-khoa/{name2}"

        # Khởi tạo driver (headless Chrome)
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(options=chrome_options, service=service)
        driver.get(dienmaycholon_url)

        # Tìm kiếm danh sách element
        name_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "h3.name_pro"))
        )

        # Tìm kiếm danh sách element giá sản phẩm
        price_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "div.price_sale"))
        )

        # Tìm kiếm danh sách element ảnh sản phẩm
        image_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "a.img_pro img"))
        )

        # # Kiểm tra số lượng element
        num_elements = len(name_elements)
        # print(num_elements)
        if num_elements != len(price_elements) or num_elements != len(image_elements):
            print("Điện máy chợ lớn: Số lượng element không khớp!")
            driver.quit()
            return []

        # Lấy danh sách sản phẩm
        products = []
        min_price = float('inf')  # Khởi tạo giá trị min_price là vô cùng
        cheapest_product = None
        for i in range(num_elements):
            # Lấy tên sản phẩm
            dienmaycholon_name = name_elements[i].text

            # Lấy giá sản phẩm
            dienmaycholon_price = price_elements[i].text.strip('đ')

            # Chuyển đổi giá thành số (xóa dấu chấm)
            try:
                price_num = int(dienmaycholon_price.replace('.', ''))
            except ValueError:
                print(f"Lỗi chuyển đổi giá: {dienmaycholon_price}")
                continue  # Bỏ qua sản phẩm này nếu không thể chuyển đổi giá

            # Lấy link ảnh sản phẩm
            image_src = image_elements[i].get_attribute("src")
            if not image_src:
                image_src = image_elements[i].get_attribute("data-src")

            # Lưu thông tin vào dictionary
            product = {
                "name": dienmaycholon_name,
                "price": dienmaycholon_price,
                "image": image_src,
                "link": dienmaycholon_url
            }

            # Kiểm tra giá và cập nhật sản phẩm rẻ nhất
            if price_num < min_price:
                min_price = price_num
                cheapest_product = product

        # Thêm sản phẩm rẻ nhất vào danh sách (nếu có)
        if cheapest_product:
            products.append(cheapest_product)

        driver.quit()  # Đóng trình duyệt

        # In thông tin sản phẩm (nếu có)
        if products:
            for product in products:
                print("Điện máy chợ lớn:")
                print("Tên Sản Phẩm:", product["name"])
                print("Giá:", product["price"])
                print("Link Ảnh:", product["image"])
                print("Link:", product["link"])
                print("---------------------------------")
        else:
            print("Điện máy chợ lớn: No product found!")
            print("---------------------------------")

        return products

    except Exception as e:
        print(f"Lỗi: {e}")
        return []


name = "Điện thoại"
products = dienmaycholon(name)

Điện máy chợ lớn:
Tên Sản Phẩm: Samsung Galaxy A03 (3+32GB)
Giá: 1.490.000
Link Ảnh: https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_29551/dien-thoai-sams_main_435_450.png.webp
Link: https://dienmaycholon.vn/tu-khoa/Điện+thoại
---------------------------------


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import ChromiumOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
# thư viện so sánh chuỗi trong python so sánh theo: difflib.SequenceMatcher(None, string1, string2).ratio()
import difflib


def dienmaycholon(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "+")
        dienmaycholon_url = f"https://dienmaycholon.vn/tu-khoa/{name2}"

        # Khởi tạo driver (headless Chrome)
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(options=chrome_options, service=service)
        driver.get(dienmaycholon_url)

        # Tìm kiếm danh sách element
        name_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "h3.name_pro"))
        )

        # Tìm kiếm danh sách element giá sản phẩm
        price_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "div.price_sale"))
        )

        # Tìm kiếm danh sách element ảnh sản phẩm
        image_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "a.img_pro img"))
        )

        # Kiểm tra số lượng element
        num_elements = len(name_elements)
        if num_elements != len(price_elements) or num_elements != len(image_elements):
            print("Điện máy chợ lớn: Số lượng element không khớp!")
            driver.quit()
            return []

        # Lấy danh sách sản phẩm
        products = []
        min_price = float('inf')  # Khởi tạo giá trị min_price là vô cùng
        cheapest_product = None
        for i in range(num_elements):
            # Lấy tên sản phẩm
            dienmaycholon_name = name_elements[i].text

            # So sánh tên sản phẩm với chuỗi đầu vào
            similarity_ratio = difflib.SequenceMatcher(
                None, name.lower(), dienmaycholon_name.lower()).ratio()
            if similarity_ratio < 0.8:  # Điều chỉnh ngưỡng 80% theo yêu cầu
                # Nếu độ tương tự thấp hơn 80%, thì lấy tên sản phẩm tiếp theo
                continue

            # Lấy giá sản phẩm
            dienmaycholon_price = price_elements[i].text.strip('đ')

            # Chuyển đổi giá thành số (xóa dấu chấm)
            try:
                price_num = int(dienmaycholon_price.replace('.', ''))
            except ValueError:
                print(f"Lỗi chuyển đổi giá: {dienmaycholon_price}")
                continue  # Bỏ qua sản phẩm này nếu không thể chuyển đổi giá

            # Lấy link ảnh sản phẩm
            image_src = image_elements[i].get_attribute("src")
            if not image_src:
                image_src = image_elements[i].get_attribute("data-src")

            # Lưu thông tin vào dictionary
            product = {
                "name": dienmaycholon_name,
                "price": dienmaycholon_price,
                "image": image_src,
                "link": dienmaycholon_url
            }

            # Kiểm tra giá và cập nhật sản phẩm rẻ nhất
            if price_num < min_price:
                min_price = price_num
                cheapest_product = product

        # Thêm sản phẩm rẻ nhất vào danh sách (nếu có)
        if cheapest_product:
            products.append(cheapest_product)

        driver.quit()  # Đóng trình duyệt

        # In thông tin sản phẩm (nếu có)
        if products:
            for product in products:
                print("Điện máy chợ lớn:")
                print("Tên Sản Phẩm:", product["name"])
                print("Giá:", product["price"])
                print("Link Ảnh:", product["image"])
                print("Link:", product["link"])
                print("---------------------------------")
        else:
            print("Điện máy chợ lớn: No product found!")
            print("---------------------------------")

        return products

    except Exception as e:
        print(f"Lỗi: {e}")
        return []


name = "Smart Tivi Samsung 4K UHD 55 Inch"
products = dienmaycholon(name)

Điện máy chợ lớn: No product found!
---------------------------------


In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import ChromiumOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.common.by import By

name = "Macbook pro"
# Chuẩn bị URL
name2 = name.replace(" ", "-")
dienmaycholon_url = f"https://dienmaycholon.vn/tu-khoa/{name2}"

# Khởi tạo driver (headless Chrome)
chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--disable-gpu")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(options=chrome_options, service=service)
driver.get(dienmaycholon_url)

In [10]:
# Tìm kiếm danh sách element
name_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3.name_pro"))
)

# Lấy danh sách tên sản phẩm
dienmaycholon_names = []
for name_element in name_elements:
    # Lọc text để lấy tên sản phẩm
    dienmaycholon_name = name_element.text
    dienmaycholon_names.append(dienmaycholon_name)

# In danh sách tên sản phẩm
print(dienmaycholon_names)

['Apple iPhone 15 Pro 128GB', 'Apple iPhone 15 Pro 256GB', 'Apple iPhone 15 Pro 512GB', 'Apple iPhone 15 Pro 1TB', 'Apple iPhone 15 Pro Max 256GB', 'Xiaomi Redmi Note 13 Pro (8GB+128GB)', 'Kalite Nồi chiên không dầu STEAM PRO', 'Xiaomi Redmi Note 13 Pro Plus 5G (8GB+256GB)', 'Apple iPhone 15 Pro Max 512GB Titan Tự Nhiên', 'Xiaomi Xiaomi Mi 10T Pro (8GB+128GB) Đen', 'Realme 11 Pro 5G (8GB+256GB)', 'Xiaomi Redmi Note 12 Pro (8GB+256GB)', 'Xiaomi Redmi Note 11 Pro 5G (8+128GB)', 'Magic Bàn Chải Điện Pro-50 Màu Xanh', 'Dreame Máy Hút Bụi Cầm Tay V12 Pro']


In [11]:
len(dienmaycholon_names)

15

In [13]:
# Tìm kiếm danh sách element giá sản phẩm
price_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.price_sale"))
)

# Lấy danh sách giá sản phẩm
dienmaycholon_prices = []
for price_element in price_elements:
    # Lấy text và xóa ký tự "đ"
    dienmaycholon_price = price_element.text.strip(
        'đ')  # Xóa dấu chấm phân cách hàng nghìn
    dienmaycholon_prices.append(dienmaycholon_price)

# In danh sách giá sản phẩm
print(dienmaycholon_prices)
print(len(dienmaycholon_prices))

['24.990.000', '27.990.000', '34.490.000', '39.990.000', '28.990.000', '6.090.000', '4.349.000', '9.490.000', '35.990.000', '8.790.000', '9.990.000', '4.290.000', '5.790.000', '990.000', 'Rẻ hơn: 10.790.000']
15


In [15]:
# Tìm kiếm danh sách element ảnh sản phẩm
image_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.img_pro img"))
)

# Lấy danh sách link ảnh sản phẩm
dienmaycholon_images = []
for image_element in image_elements:
    # Lấy thuộc tính src hoặc data-src
    image_src = image_element.get_attribute("src")
    if not image_src:  # Nếu src trống, lấy data-src
        image_src = image_element.get_attribute("data-src")
    dienmaycholon_images.append(image_src)

# In danh sách link ảnh sản phẩm
print(dienmaycholon_images)
print(len(dienmaycholon_images))

['https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_33048/iphone-15-pro-1_main_288_450.png.webp', 'https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_33380/iphone-15-pro-2_main_863_450.png.webp', 'https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_33381/iphone-15-pro-5_main_343_450.png.webp', 'https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_33382/iphone-15-pro-1_main_185_450.png.webp', 'https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_33049/iphone-15-pro-m_main_61_450.png.webp', 'https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_34178/xiaomi-redmi-no_main_363_450.png.webp', 'https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_29758/noi-chien-khong_main_429_450.png.webp', 'https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_34050/xiaomi-redmi-no_main_937

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import ChromiumOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.common.by import By


def dienmaycholon(name):

    try:

        # Chuẩn bị URL

        name2 = name.replace(" ", "+")

        dienmaycholon_url = f"https://dienmaycholon.vn/tu-khoa/{name2}"



        # Khởi tạo options cho Chrome

        chrome_options = Options()

        chrome_options.add_argument("--headless")  # Chạy headless

        chrome_options.add_argument("--disable-gpu")  # Tắt GPU

        chrome_options.add_experimental_option(
            'useAutomationExtension', False)  # Tắt thông báo Chrome Automation

        chrome_options.add_argument(
            '--ignore-certificate-errors')  # Bỏ qua lỗi certificate

        chrome_options.add_argument('--start-maximized')  # Mở toàn màn hình



        # Khởi tạo driver

        service = Service(ChromeDriverManager().install())

        driver = webdriver.Chrome(options=chrome_options, service=service)

        driver.get(dienmaycholon_url)

        # driver.implicitly_wait(10)  # Chờ đợi element xuất hiện



        # Tìm kiếm element

        try:

            element_name = WebDriverWait(driver, 10).until(

                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "h3.name_pro"))

            )

            element_price = WebDriverWait(driver, 10).until(

                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "div.price_sale"))

            )

            element_image = WebDriverWait(driver, 10).until(

                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "a.img_pro img"))

            )

        except TimeoutException:

            driver.quit()

            return "0", "0", "0", "0"



        # Lấy thông tin sản phẩm

        dienmaycholon_name = element_name.text

        dienmaycholon_price = element_price.text.strip('đ')

        dienmaycholon_image = element_image.get_attribute("src")



        driver.quit()  # Đóng trình duyệt



        return dienmaycholon_price, dienmaycholon_name[0:50], dienmaycholon_image, dienmaycholon_url



    except Exception as e:

        print(f"Lỗi: {e}")

        return "0", "0", "0", "0"





dienmaycholon("iphone 15")

('18.990.000',
 'Apple iPhone 15 128GB',
 'https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_33035/iphone-15-128gb_main_586_450.png.webp',
 'https://dienmaycholon.vn/tu-khoa/iphone+15')

In [8]:

# *************************Điện máy chợ lớn*************************
def dienmaycholon(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "-")
        dienmaycholon_url = f"https://dienmaycholon.vn/tu-khoa/{name2}"
        print(dienmaycholon_url)
        
        # Khởi tạo options cho Chrome
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # Chạy headless
        chrome_options.add_argument("--disable-gpu")  # Tắt GPU
        # chrome_options.add_experimental_option('useAutomationExtension', False) # Tắt thông báo Chrome Automation
        # chrome_options.add_argument('--ignore-certificate-errors') # Bỏ qua lỗi certificate
        # chrome_options.add_argument('--start-maximized') # Mở toàn màn hình

        # Khởi tạo driver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(options=chrome_options, service=service)
        driver.get(dienmaycholon_url)
        driver.implicitly_wait(5)  # Chờ đợi element xuất hiện
        print("---------------------------------")
        print("\nSearching in Điện Máy Chợ Lớn...")

        # Tìm kiếm element
        try:
            element_name = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "h3.name_pro"))
            )
            element_price = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "div.price_sale"))
            )
            element_image = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "a.img_pro img"))
            )
        except TimeoutException:
            driver.quit()
            return "0", "0", "0", "0"

        # Lấy thông tin sản phẩm
        dienmaycholon_name = element_name.text
        # print(dienmaycholon_name)

        dienmaycholon_price = element_price.text.strip('đ')
        # Rẻhơn:8490000
        # xử lý giá nếu có dạng "Rẻ hơn:8490000" thì chỉ lấy giá ở sau dấu ":"
        dienmaycholon_price = dienmaycholon_price.split(":")[-1]
        # print(dienmaycholon_price)

        dienmaycholon_image = element_image.get_attribute("src")

        driver.quit()  # Đóng trình duyệt
        print("Điện Máy Chợ Lớn:")
        print("Tên Sản Phẩm:", dienmaycholon_name)
        print("Giá:", dienmaycholon_price)
        print("Link Ảnh:", dienmaycholon_image)
        print("Link:", dienmaycholon_url)
        print("---------------------------------")

        return dienmaycholon_price, dienmaycholon_name[0:50], dienmaycholon_image, dienmaycholon_url

    except Exception as e:
        print(f"Lỗi: {e}")
        dienmaycholon_price = '0'
        dienmaycholon_name = '0'
        dienmaycholon_image = '0'
        dienmaycholon_url = '0'
        return dienmaycholon_price, dienmaycholon_name, dienmaycholon_image, dienmaycholon_url


name = "iPhone 15 Pro"
dienmaycholon(name)
# dienmaycholon_price, dienmaycholon_name, dienmaycholon_image, dienmaycholon_url

https://dienmaycholon.vn/tu-khoa/iPhone-15-Pro
---------------------------------

Searching in Điện Máy Chợ Lớn...
Điện Máy Chợ Lớn:
Tên Sản Phẩm: Apple iPhone 15 Pro 128GB
Giá: 24.990.000
Link Ảnh: https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_33048/iphone-15-pro-1_main_288_450.png.webp
Link: https://dienmaycholon.vn/tu-khoa/iPhone-15-Pro
---------------------------------


('24.990.000',
 'Apple iPhone 15 Pro 128GB',
 'https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_33048/iphone-15-pro-1_main_288_450.png.webp',
 'https://dienmaycholon.vn/tu-khoa/iPhone-15-Pro')

---

In [11]:
import imp
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pathlib import Path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from django.conf import settings
import re

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

from fuzzywuzzy import fuzz

# ****************************************Format**********************************************

from deep_translator import GoogleTranslator
from selenium.common.exceptions import NoSuchElementException

In [8]:
def sendo(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "+")
        sendo_url = f"https://www.sendo.vn/tim-kiem?q={name2}"

        # Khởi tạo driver (headless Chrome)
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(options=chrome_options, service=service)
        driver.get(sendo_url)        
        
        # Tìm kiếm danh sách element
        name_elements = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.d7ed-Vp2Ugh._0032-Zwkt7j"))
                         )
        # Tìm kiếm danh sách element giá sản phẩm
        price_elements = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span._0032-GpBMYp._0032-npoTU_.d7ed-CLUDGW.d7ed-AHa8cD.d7ed-giDKVr"))
                        )
        # Tìm kiếm danh sách element ảnh sản phẩm
        image_elements = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.d7ed-a1ulZz img"))
                        )

        # # Kiểm tra số lượng element
        num_elements = len(name_elements)
        # print(num_elements)
        if num_elements != len(price_elements) or num_elements != len(image_elements):
            print("sendo: Số lượng element không khớp!")
            driver.quit()
            return []

        # Lấy danh sách sản phẩm
        products = []

        for i in range(num_elements):
            similarity_score = fuzz.ratio(name.upper(), name_elements[i].text.upper())
            if similarity_score >= 10:
                # Lấy tên sản phẩm
                sendo_name = name_elements[i].text

                # Lấy giá sản phẩm
                sendo_price = price_elements[i].text.strip('đ')
                # '900000đ-999000 xử lý chỉ lấy giá ở sau dấu "-"
                sendo_price = sendo_price.split("-")[-1]
                sendo_price = sendo_price.replace(".", "")
                               
                # Lấy link ảnh sản phẩm
                image_src = image_elements[i].get_attribute("data-src")
                if not image_src:
                    image_src = image_elements[i].get_attribute("src")

                # Lưu thông tin vào dictionary
                product = {
                    "name": sendo_name,
                    "price": sendo_price,
                    "image": image_src,
                    "link": sendo_url,
                }

                products.append(product)
        
        driver.quit()  # Đóng trình duyệt

        
        dprice = min(products, key=lambda x: float(x['price']))
        # print(dprice)
        dprice["web_name"]="Sendo"
        
        # Lấy sản phẩm đầu
        # sendo_name = products[0]["name"]
        # sendo_price = products[0]["price"]  
        # sendo_image = products[0]["image"]  
        # sendo_link = products[0]["link"]
        
        if products:
            # print("Sendo:")
            # print("Tên Sản Phẩm:", sendo_name)
            # print("Giá:", sendo_price)
            # print("Link Ảnh:", sendo_image)
            # print("Link:", sendo_link)
            # print("---------------------------------")
            print("Sendo:")
            print("Tên Sản Phẩm:", dprice["name"])
            print("Giá:", dprice["price"])
            print("Link Ảnh:", dprice["image"])
            print("Link:", dprice["link"])
            print("---------------------------------")
            return dprice           
        else:   
            print("sendo: No product found!")
            print("---------------------------------")
            return {}
                                         
    except Exception as e:
        print(f"Lỗi: {e}")
        return {}

In [9]:
sendo("Iphone")

Sendo:
Tên Sản Phẩm: iPhone 12 64GB - Xanh lá
Giá: 11990000
Link Ảnh: https://media3.scdn.vn/img4/2022/03_17/SK1OnxMmksfDKQrR1UyC_simg_de2fe0_250x250_maxb.jpg
Link: https://www.sendo.vn/tim-kiem?q=Iphone
---------------------------------


{'name': 'iPhone 12 64GB - Xanh lá',
 'price': '11990000',
 'image': 'https://media3.scdn.vn/img4/2022/03_17/SK1OnxMmksfDKQrR1UyC_simg_de2fe0_250x250_maxb.jpg',
 'link': 'https://www.sendo.vn/tim-kiem?q=Iphone',
 'web_name': 'Sendo'}

In [ ]:
# ****************************************Điện Máy Xanh*****************************************
def dienmayxanh(name):
    try:
        name2 = name.replace(" ", "+")
        dienmayxanh_url = f'https://www.dienmayxanh.com/tim-kiem?key={name2}'

        res = requests.get(dienmayxanh_url, headers=headers)
        print("\nSearching in Điện Máy Xanh...")
        soup = BeautifulSoup(res.text, 'html.parser')

        dienmayxanh_page = soup.select('a.main-contain')
        dienmayxanh_page_length = int(len(dienmayxanh_page))        
        
        matching_products = []  # List to store matching products
        for i in range(0, dienmayxanh_page_length):
            dienmayxanh_name = soup.select('a.main-contain>h3')[i].getText().strip().upper()
   
            similarity_score = fuzz.ratio(name.upper(), dienmayxanh_name)
            # print(f"Similarity Score: {similarity_score}")
                
            if similarity_score >= 10:
                dienmayxanh_name = soup.select('a.main-contain>h3')[i].getText().strip()
                # print(f"Name: {dienmayxanh_name}")

                dienmayxanh_images = soup.select('a.main-contain')
                if dienmayxanh_images:
                    if dienmayxanh_images[i].find('img').get('src'):
                        dienmayxanh_image = dienmayxanh_images[i].find('img')['src']
                    elif dienmayxanh_images[i].find_all('img', class_='lazyload'):
                        dienmayxanh_image = dienmayxanh_images[i].find_all('img', class_='lazyload')[0]['data-src']
                    elif dienmayxanh_images[i].find_all('img', class_='lazyloaded'):
                        dienmayxanh_image = dienmayxanh_images[i].find_all('img', class_='lazyloaded')[0]['data-src']  
                    else:
                        dienmayxanh_image = '0'
                else:
                    dienmayxanh_image = '0'
                
                try:
                    # Robust empty price check
                    if not soup.select('a.main-contain>strong.price')[i].getText().strip() or soup.select('a.main-contain>strong.price')[i].getText().strip() == ' ':
                        dienmayxanh_price = '0'
                    else:
                        dienmayxanh_price = soup.select('a.main-contain>strong.price')[i].getText().strip().upper()
                        dienmayxanh_price = dienmayxanh_price.strip('₫')
                        dienmayxanh_price = re.sub("[^0-9]", "", dienmayxanh_price)
                except:
                    continue        

                product = {
                    "name": dienmayxanh_name,
                    "price": dienmayxanh_price,
                    "image": dienmayxanh_image,
                    "link": dienmayxanh_url
                }

                # Kiểm tra xem sản phẩm nào price=0 thì không lưu vào matching_products và xoá product đó
                if product["price"] == '0':
                    del product
                else:
                    matching_products.append(product)
          
        # Nếu không có sản phẩm nào thì trả về None
        if not matching_products:
            return None  
      
        dienmayxanh_price = matching_products[0]["price"]
        dienmayxanh_name= matching_products[0]["name"]
        dienmayxanh_image= matching_products[0]["image"]
        dienmayxanh_url= matching_products[0]["link"]
        print("Điện Máy Xanh:")
        print("Tên Sản Phẩm:", dienmayxanh_name)
        print("Giá:", dienmayxanh_price)
        print("Link Ảnh:", dienmayxanh_image)
        print("Link:", dienmayxanh_url)
        print("---------------------------------")
    
        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url

    except Exception as e:
        print(f"Lỗi: {e}")
        print("Điện Máy Xanh: No product found!")
        print("---------------------------------")
        dienmayxanh_price = '0'
        dienmayxanh_name = '0'
        dienmayxanh_image = '0'
        dienmayxanh_url = '0'
        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url

In [ ]:
#********************************************Chợ tốt**********************************************************************
def chotot(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "+")
        chotot_url = f'https://www.chotot.com/mua-ban?q={name2}'

        # Thiết lập options cho Chrome headless
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")

        # Khởi tạo driver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(options=chrome_options, service=service)
        driver.get(chotot_url)

        # Tìm kiếm danh sách element sản phẩm
        product_elements = driver.find_elements(By.CSS_SELECTOR, "div.ListAds_ListAds__rEu_9")

        products = []
        for product_element in product_elements:
            # Lấy thông tin sản phẩm
            try:
                name_element = product_element.find_element(By.CSS_SELECTOR, "h3.commonStyle_adTitle__g520j")
                price_element = product_element.find_element(By.CSS_SELECTOR, "p.AdBody_adPriceNormal___OYFU")
                image_element = product_element.find_element(By.CSS_SELECTOR, "picture.webpimg-container img")

                chotot_name = name_element.text
                chotot_price = price_element.text.strip('đ')
                chotot_image = image_element.get_attribute("src")

                # Lưu thông tin vào dictionary
                product = {
                    "name": chotot_name,
                    "price": chotot_price,
                    "image": chotot_image,
                    "link": chotot_url
                }
                products.append(product)
            except NoSuchElementException:
                # Bỏ qua nếu element không tồn tại
                pass

        driver.quit()  # Đóng trình duyệt

        # In thông tin sản phẩm (nếu có)
        if products:
            for product in products:
                print("Chợ tốt:")
                print("Tên Sản Phẩm:", product["name"])
                print("Giá:", product["price"])
                print("Link Ảnh:", product["image"])
                print("Link:", product["link"])
                print("---------------------------------")
        else:
            print("chotot: No product found!")
            print("---------------------------------")

        return products

    except Exception as e:
        print(f"Lỗi: {e}")
        return []

In [10]:
driver.get("https://www.sendo.vn/tim-kiem?q=iphone")

NameError: name 'driver' is not defined

In [5]:
import imp
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pathlib import Path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from django.conf import settings
import re

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

from fuzzywuzzy import fuzz


# Thiết lập options cho Chrome headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

# Khởi tạo driver
service = Service(ChromeDriverManager().install())
sendo_driver = webdriver.Chrome(options=chrome_options, service=service)
dienmaycholon_driver = webdriver.Chrome(options=chrome_options, service=service)
chotot_driver = webdriver.Chrome(options=chrome_options, service=service)

# ****************************************Format**********************************************

from deep_translator import GoogleTranslator
from selenium.common.exceptions import NoSuchElementException

def translator(text):
    """
    Dịch văn bản từ ngôn ngữ hiện tại sang tiếng Anh.
    # Đối với các trang quốc tế: Amazon, ...
    """
    
    type_trans = GoogleTranslator(source='auto', target='en')
    trans = type_trans.translate(text)
    return trans

def no_accent_vietnamese(s):
    s = s.lower()
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('đ', 'd', s)
    return s

def convert(a):
    b = a.replace(" ", '')
    c = b.replace("INR", '')
    d = c.replace(",", '')
    d = d.replace("`", '')
    f = d.replace("₹", '')
    f1 = f.replace(".", '')
    f2 = f1.replace("₫", '')
    g = int(float(f2))
    return g

def format_price(price):
    """Formats a price number into a string with commas, currency symbol, and handles potential decimals.

  Args:
      price: The price as a number (float or int).

  Returns:
      A string representing the formatted price.
  """
    # Handle potential decimals
    if isinstance(price, float):
        price_str = "{:,.2f}".format(price)  # Two decimal places for floats
    else:
        price_str = "{:,}".format(price)  # No decimal places for integers

    return price_str + "đ"  # Add currency symbol (đ)
# **********************************************************************************************






# ****************************************Điện Máy Xanh*****************************************
def dienmayxanh(name):
    try:
        name2 = name.replace(" ", "+")
        dienmayxanh_url = f'https://www.dienmayxanh.com/tim-kiem?key={name2}'

        res = requests.get(dienmayxanh_url, headers=headers)
        print("\nSearching in Điện Máy Xanh...")
        soup = BeautifulSoup(res.text, 'html.parser')

        dienmayxanh_page = soup.select('a.main-contain')
        dienmayxanh_page_length = int(len(dienmayxanh_page))        
        
        matching_products = []  # List to store matching products
        for i in range(0, dienmayxanh_page_length):
            dienmayxanh_name = soup.select('a.main-contain>h3')[i].getText().strip().upper()
   
            similarity_score = fuzz.ratio(name.upper(), dienmayxanh_name)
            # print(f"Similarity Score: {similarity_score}")
                
            if similarity_score >= 10:
                dienmayxanh_name = soup.select('a.main-contain>h3')[i].getText().strip()
                # print(f"Name: {dienmayxanh_name}")

                dienmayxanh_images = soup.select('a.main-contain')
                if dienmayxanh_images:
                    if dienmayxanh_images[i].find('img').get('src'):
                        dienmayxanh_image = dienmayxanh_images[i].find('img')['src']
                    elif dienmayxanh_images[i].find_all('img', class_='lazyload'):
                        dienmayxanh_image = dienmayxanh_images[i].find_all('img', class_='lazyload')[0]['data-src']
                    elif dienmayxanh_images[i].find_all('img', class_='lazyloaded'):
                        dienmayxanh_image = dienmayxanh_images[i].find_all('img', class_='lazyloaded')[0]['data-src']  
                    else:
                        dienmayxanh_image = '0'
                else:
                    dienmayxanh_image = '0'
                
                try:
                    # Robust empty price check
                    if not soup.select('a.main-contain>strong.price')[i].getText().strip() or soup.select('a.main-contain>strong.price')[i].getText().strip() == ' ':
                        dienmayxanh_price = '0'
                    else:
                        dienmayxanh_price = soup.select('a.main-contain>strong.price')[i].getText().strip().upper()
                        dienmayxanh_price = dienmayxanh_price.strip('₫')
                        dienmayxanh_price = re.sub("[^0-9]", "", dienmayxanh_price)
                except:
                    continue        

                product = {
                    "name": dienmayxanh_name,
                    "price": dienmayxanh_price,
                    "image": dienmayxanh_image,
                    "link": dienmayxanh_url
                }

                # Kiểm tra xem sản phẩm nào price=0 thì không lưu vào matching_products và xoá product đó
                if product["price"] == '0':
                    del product
                else:
                    matching_products.append(product)
          
        # Nếu không có sản phẩm nào thì trả về None
        if not matching_products:
            return None  
        
        #------------------------------------------------------------------------
        # print("Matching Products:")
        # for product in matching_products:
        #     print("Tên Sản Phẩm:", product["name"])
        #     print("Giá:", product["price"])
        #     print("Link Ảnh:", product["image"])
        #     print("Link:", product["link"])
        #     print("---------------------------------")
        
        # # Find the cheapest product
        # cheapest_product = min(matching_products, key=lambda p: int(p["price"].replace('.', '')))
                
        # if cheapest_product:
        #     print("Tên Sản Phẩm:", cheapest_product["name"])
        #     print("Giá:", cheapest_product["price"])
        #     print("Link Ảnh:", cheapest_product["image"])
        #     print("Link:", cheapest_product["link"])
        #     print("---------------------------------") 
        # else:
        #     print("No matching product found.")    
        
        # Lưu cheapest_product vào list_product
        # list_product = []
        # list_product.append(cheapest_product)
        #------------------------------------------------------------------------
      
        dienmayxanh_price = matching_products[0]["price"]
        dienmayxanh_name= matching_products[0]["name"]
        dienmayxanh_image= matching_products[0]["image"]
        dienmayxanh_url= matching_products[0]["link"]
        print("Điện Máy Xanh:")
        print("Tên Sản Phẩm:", dienmayxanh_name)
        print("Giá:", dienmayxanh_price)
        print("Link Ảnh:", dienmayxanh_image)
        print("Link:", dienmayxanh_url)
        print("---------------------------------")
    
        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url

    except Exception as e:
        print(f"Lỗi: {e}")
        print("Điện Máy Xanh: No product found!")
        print("---------------------------------")
        dienmayxanh_price = '0'
        dienmayxanh_name = '0'
        dienmayxanh_image = '0'
        dienmayxanh_url = '0'
        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url
# ******************************************************************************************************


#  ****************************************Amazon**********************************************
def amazon(name):
    try:
        global amazon
        name = translator(name)
        name1 = name.replace(" ", "-")
        name2 = name.replace(" ", "+")
        amazon = f'https://www.amazon.in/{name1}/s?k={name2}'
        amazon_link = amazon
        print(amazon_link)
        res = requests.get(
            f'https://www.amazon.in/{name1}/s?k={name2}', headers=headers)
        print("\nSearching in amazon...")
        soup = BeautifulSoup(res.text, 'html.parser')
        amazon_page = soup.select('.a-color-base.a-text-normal')
        amazon_page_length = int(len(amazon_page))
        for i in range(0, amazon_page_length):
            name = name.upper()
            amazon_name = soup.select(
                '.a-color-base.a-text-normal')[i].getText().strip().upper()
            
            # Tính toán độ tương đồng giữa tên sản phẩm và tên sản phẩm tìm kiếm
            similarity_score = fuzz.ratio(name, amazon_name)
            # print(f"Similarity Score: {similarity_score}")
            if similarity_score >= 10:
                amazon_name = soup.select(
                    '.a-color-base.a-text-normal')[i].getText().strip()
                amazon_images = soup.select(
                    '.a-section.aok-relative.s-image-fixed-height')
                amazon_image = amazon_images[0].find_all(
                    'img', class_='s-image')[0]
                amazon_image = amazon_image['src']
                amazon_price = soup.select(
                    '.a-price-whole')[i].getText().strip().upper()
                print("Amazon:")
                print("Tên Sản Phẩm:", amazon_name)
                print("Giá:", amazon_price)
                print("Link Ảnh:", amazon_image)
                print("Link:", amazon_link)
                print("---------------------------------")
                break
            else:
                i += 1
                i = int(i)
                if i == amazon_page_length:
                    amazon_price = '0'
                    print("amazon : No product found!")
                    print("-----------------------------")
                    break

        return amazon_price, amazon_name[0:50], amazon_image, amazon_link
    except:
        print("Amazon: No product found!")
        print("---------------------------------")
        amazon_price = '0'
        amazon_name = '0'
        amazon_link = '0'
        amazon_image = '0'
    return amazon_price, amazon_name[0:50], amazon_image, amazon_link
# ====================================================================================


# 



#********************************************Chợ tốt**********************************************************************

def chotot(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "+")
        chotot_url = f'https://www.chotot.com/mua-ban?q={name2}'
        
        chotot_driver.get(chotot_url)

        # Tìm kiếm danh sách element sản phẩm
        product_elements = chotot_driver.find_elements(By.CSS_SELECTOR, "div.ListAds_ListAds__rEu_9")

        products = []
        for product_element in product_elements:
            # Lấy thông tin sản phẩm
            try:
                name_element = product_element.find_element(By.CSS_SELECTOR, "h3.commonStyle_adTitle__g520j")
                price_element = product_element.find_element(By.CSS_SELECTOR, "p.AdBody_adPriceNormal___OYFU")
                image_element = product_element.find_element(By.CSS_SELECTOR, "picture.webpimg-container img")

                chotot_name = name_element.text
                chotot_price = price_element.text.strip('đ')
                chotot_image = image_element.get_attribute("src")

                # Lưu thông tin vào dictionary
                product = {
                    "name": chotot_name,
                    "price": chotot_price,
                    "image": chotot_image,
                    "link": chotot_url
                }
                products.append(product)
            except NoSuchElementException:
                # Bỏ qua nếu element không tồn tại
                pass

        chotot_driver.quit()  # Đóng trình duyệt

        # In thông tin sản phẩm (nếu có)
        if products:
            for product in products:
                print("Chợ tốt:")
                print("Tên Sản Phẩm:", product["name"])
                print("Giá:", product["price"])
                print("Link Ảnh:", product["image"])
                print("Link:", product["link"])
                print("---------------------------------")
        else:
            print("chotot: No product found!")
            print("---------------------------------")

        return products

    except Exception as e:
        print(f"Lỗi: {e}")
        return []
# *********************************************************************************************************

sendo_driver = webdriver.Chrome(options=chrome_options, service=service)
def sendo(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "+")
        sendo_url = f"https://www.sendo.vn/tim-kiem?q={name2}"

        sendo_driver.get(sendo_url)        
        
        # Tìm kiếm danh sách element
        name_elements = WebDriverWait(sendo_driver, 10).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.d7ed-Vp2Ugh._0032-Zwkt7j"))
                         )
        # Tìm kiếm danh sách element giá sản phẩm
        price_elements = WebDriverWait(sendo_driver, 10).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span._0032-GpBMYp._0032-npoTU_.d7ed-CLUDGW.d7ed-AHa8cD.d7ed-giDKVr"))
                        )
        # Tìm kiếm danh sách element ảnh sản phẩm
        image_elements = WebDriverWait(sendo_driver, 10).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.d7ed-a1ulZz img"))
                        )

        # # Kiểm tra số lượng element
        num_elements = len(name_elements)
        # print(num_elements)
        if num_elements != len(price_elements) or num_elements != len(image_elements):
            print("sendo: Số lượng element không khớp!")
            sendo_driver.quit()
            return []

        # Lấy danh sách sản phẩm
        products = []

        for i in range(num_elements):
            similarity_score = fuzz.ratio(name.upper(), name_elements[i].text.upper())
            if similarity_score >= 10:
                # Lấy tên sản phẩm
                sendo_name = name_elements[i].text

                # Lấy giá sản phẩm
                sendo_price = price_elements[i].text.strip('đ')
                # '900000đ-999000 xử lý chỉ lấy giá ở sau dấu "-"
                sendo_price = sendo_price.split("-")[-1]
                sendo_price = sendo_price.replace(".", "")
                                

                # Lấy link ảnh sản phẩm
                image_src = image_elements[i].get_attribute("data-src")
                if not image_src:
                    image_src = image_elements[i].get_attribute("src")

                # Lưu thông tin vào dictionary
                product = {
                    "name": sendo_name,
                    "price": sendo_price,
                    "image": image_src,
                    "link": sendo_url
                }

                products.append(product)
        
        sendo_driver.quit()  # Đóng trình duyệt

        # Lấy sản phẩm đầu
        sendo_name = products[0]["name"]
        sendo_price = products[0]["price"]  
        sendo_image = products[0]["image"]  
        sendo_link = products[0]["link"]
        
        if products:
            print("Sendo:")
            print("Tên Sản Phẩm:", sendo_name)
            print("Giá:", sendo_price)
            print("Link Ảnh:", sendo_image)
            print("Link:", sendo_link)
            print("---------------------------------")
            return sendo_price, sendo_name[0:50], sendo_image, sendo_link           
        else:   
            print("sendo: No product found!")
            print("---------------------------------")
            sendo_price = '0'   
            sendo_name = '0'
            sendo_link = '0'
            sendo_image = '0'
            return sendo_price, sendo_name, sendo_image, sendo_link
                                         
    except Exception as e:
        print(f"Lỗi: {e}")
        sendo_price = '0'
        sendo_name = '0'
        sendo_link = '0'
        sendo_image = '0'
        return sendo_price, sendo_name, sendo_image, sendo_link
    
# **************************************************************************

# *************************Điện máy chợ lớn*************************
def dienmaycholon(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "-")
        dienmaycholon_url = f"https://dienmaycholon.vn/tu-khoa/{name2}"

        dienmaycholon_driver.get(dienmaycholon_url) 
        # driver.implicitly_wait(10)  # Chờ đợi element xuất hiện 
        print("---------------------------------")
        print("\nSearching in Điện Máy Chợ Lớn...")
        
        # Tìm kiếm element
        try:
            element_name = WebDriverWait(dienmaycholon_driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h3.name_pro"))
            )
            element_price = WebDriverWait(dienmaycholon_driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.price_sale"))
            )
            element_image = WebDriverWait(dienmaycholon_driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a.img_pro img"))
            )
        except TimeoutException:
            dienmaycholon_driver.quit()
            return "0", "0", "0", "0"

        # Lấy thông tin sản phẩm
        dienmaycholon_name = element_name.text
        dienmaycholon_price = element_price.text.strip('đ')
        # Rẻhơn:8490000 
        # xử lý giá nếu có dạng "Rẻ hơn:8490000" thì chỉ lấy giá ở sau dấu ":"
        dienmaycholon_price = dienmaycholon_price.split(":")[-1]
                
        dienmaycholon_image = element_image.get_attribute("src")
        
        dienmaycholon_driver.quit()  # Đóng trình duyệt
        print("Điện Máy Chợ Lớn:")
        print("Tên Sản Phẩm:", dienmaycholon_name)
        print("Giá:", dienmaycholon_price)
        print("Link Ảnh:", dienmaycholon_image)
        print("Link:", dienmaycholon_url)
        print("---------------------------------")

        return dienmaycholon_price, dienmaycholon_name[0:50], dienmaycholon_image, dienmaycholon_url

    except Exception as e:
        print(f"Lỗi: {e}")
        dienmaycholon_price = '0'
        dienmaycholon_name = '0'
        dienmaycholon_image = '0'
        dienmaycholon_url = '0'
        return dienmaycholon_price, dienmaycholon_name, dienmaycholon_image, dienmaycholon_url
    
# ******************************************

def croma(name):
    try:
        global croma
        name1 = name.replace(" ", "-")
        name2 = name.replace(" ", "+")
        croma = f"https://www.croma.com/search/?q={name2}:relevance:ZAStatusFlag:true:excludeOOSFlag&text={name2}"
        source = croma
        croma_link = croma
        wait_imp = 10
        CO = webdriver.ChromeOptions()
        CO.add_experimental_option('useAutomationExtension', False)
        CO.add_argument('--ignore-certificate-errors')
        CO.add_argument('--start-maximized')
        # print("Driver path", str(settings.BASE_DIR)+'\chromedriver.exe')
        wd = webdriver.Chrome('chromedriver.exe', options=CO)

        wd.get(source)
        wd.implicitly_wait(wait_imp)

        try:
            elementname = WebDriverWait(wd, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "h3.product-title.plp-prod-title"))
            )
            elementprice = WebDriverWait(wd, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "span.amount"))
            )
            imgelement = WebDriverWait(wd, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "div.product-img.plp-card-thumbnail img"))
            )
        except:
            wd.quit()

        croma_name = elementname.text
        croma_price = elementprice.text
        croma_image = imgelement.get_attribute("src")
        return croma_price, croma_name[0:50], croma_image, croma_link
    except:
        print("Croma: No product found!")
        print("---------------------------------")
        croma_price = '0'
        croma_name = '0'
        croma_image = '0'
        croma_link = '0'
    return croma_price, croma_name[0:50], croma_image, croma_link


In [39]:
a = ""
b = "345"

a or b

'345'

In [41]:

from bs4 import BeautifulSoup
import numpy as np


In [42]:
dienmaycholon_driver = webdriver.Chrome(options=chrome_options, service=service)

In [66]:
def check_val_float(val):
    try:
        float(val)
        return True
    except ValueError:
        return False

# *************************Điện máy chợ lớn*************************
def dienmaycholon(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "-")
        dienmaycholon_url = f"https://dienmaycholon.vn/tu-khoa/{name2}"
        # print(dienmaycholon_url)
        dienmaycholon_driver.get(dienmaycholon_url)
        
         
        dienmaycholon_driver.implicitly_wait(10)  # Chờ đợi element xuất hiện 
        print("---------------------------------")
        print("\nSearching in Điện Máy Chợ Lớn...")
        soup = BeautifulSoup(dienmaycholon_driver.page_source, 'lxml')
        
        # Tìm kiếm element
        element_name = soup.select("h3.name_pro")
        if len(element_name) == 0:
            raise Exception("Không tìm thấy sản phẩm!")
        
        element_price = soup.select("div.price_sale")
        
        element_image = soup.select("a.img_pro img")


        # Lấy thông tin sản phẩm   
               
        # Giá
        dienmaycholon_price = [ele.get_text().strip('đ').split(":")[-1].strip().replace(".", "") for ele in element_price]
        # dienmaycholon_price_float = [float(ele) for ele in dienmaycholon_price if check_val_float(ele)]
        
        dienmaycholon_check_vald_product = [check_val_float(ele) for ele in dienmaycholon_price]
        if not any(dienmaycholon_check_vald_product):
            raise Exception("Không tìm thấy sản phẩm!")
    
        # Lọc ra những giá trị không hợp lệ

        element_name = [ele for ele, check in zip(element_name, dienmaycholon_check_vald_product) if check]
        dienmaycholon_price = [ele for ele, check in zip(dienmaycholon_price, dienmaycholon_check_vald_product) if check]        
        element_image = [ele for ele, check in zip(element_image, dienmaycholon_check_vald_product) if check]
         
        dienmaycholon_price_float = [float(ele) for ele in dienmaycholon_price]        
        thresh = np.quantile(dienmaycholon_price_float, q = 0.05)
        # remome thresh 
        dienmaycholon_price = [ele for ele, price_float  in zip(dienmaycholon_price, dienmaycholon_price_float) if price_float>=thresh]    
        
        # Name
        # ' '.join(element_names[0].get_text().replace("\n", "").split())
        dienmaycholon_name = [' '.join(ele.get_text().replace("\n", "").split()) for ele, price_float in zip(element_name, dienmaycholon_price_float) if price_float>=thresh]
                
        # Image
        # element_image[0].attrs['src']
        dienmaycholon_image = ["https:"+ele.attrs['src'] for ele, price_float in zip(element_image, dienmaycholon_price_float) if price_float>=thresh]
        
        # Lọc giá bé nhất
        dienmaycholon_price_float = [ele for ele in dienmaycholon_price_float if ele>=thresh]
        # min(enumerate(a), key=lambda x: x[1])[0]
        index_min = min(enumerate(dienmaycholon_price_float), key=lambda x: x[1])[0]
        
        dienmaycholon_name = dienmaycholon_name[index_min]
        dienmaycholon_price = dienmaycholon_price[index_min]
        dienmaycholon_image = dienmaycholon_image[index_min]
        
        print("Điện Máy Chợ Lớn:")
        print("Tên Sản Phẩm:", dienmaycholon_name)
        print("Giá:", dienmaycholon_price)
        print("Link Ảnh:", dienmaycholon_image)
        print("Link:", dienmaycholon_url)
        print("---------------------------------")

        return dienmaycholon_price, dienmaycholon_name[0:50], dienmaycholon_image, dienmaycholon_url

    except Exception as e:
        print(f"Lỗi: {e}")
        dienmaycholon_price = '0'
        dienmaycholon_name = '0'
        dienmaycholon_image = '0'
        dienmaycholon_url = '0'
        return dienmaycholon_price, dienmaycholon_name, dienmaycholon_image, dienmaycholon_url
    
# ******************************************

In [67]:
dienmaycholon("USB")

---------------------------------

Searching in Điện Máy Chợ Lớn...
Điện Máy Chợ Lớn:
Tên Sản Phẩm: Apacer USB AH334-8GB
Giá: 77900
Link Ảnh: https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_29071/usb-apacer-ah33_main_119_450.png.webp
Link: https://dienmaycholon.vn/tu-khoa/USB
---------------------------------


('77900',
 'Apacer USB AH334-8GB',
 'https://cdn11.dienmaycholon.vn/filewebdmclnew/DMCL21/Picture//Apro/Apro_product_29071/usb-apacer-ah33_main_119_450.png.webp',
 'https://dienmaycholon.vn/tu-khoa/USB')

In [1]:
import imp
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pathlib import Path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from django.conf import settings
import re

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import numpy as np

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

from fuzzywuzzy import fuzz

C:\Users\truon\AppData\Local\Temp\ipykernel_13344\2430897423.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
d:\anaconda3\envs\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [64]:
# ****************************************Điện Máy Xanh*****************************************
def dienmayxanh(name):
    try:
        name2 = name.replace(" ", "+")
        dienmayxanh_url = f'https://www.dienmayxanh.com/tim-kiem?key={name2}'

        res = requests.get(dienmayxanh_url, headers=headers)
        print("\nSearching in Điện Máy Xanh...")
        soup = BeautifulSoup(res.text, 'html.parser')

        dienmayxanh_page = soup.select('a.main-contain')
        dienmayxanh_page_length = int(len(dienmayxanh_page))        
        
        matching_products = []  # List to store matching products
        for i in range(0, dienmayxanh_page_length):
            dienmayxanh_name = soup.select('a.main-contain>h3')[i].getText().strip().upper()
   
            similarity_score = fuzz.ratio(name.upper(), dienmayxanh_name)
            # print(f"Similarity Score: {similarity_score}")
                
            if similarity_score >= 10:
                dienmayxanh_name = soup.select('a.main-contain>h3')[i].getText().strip()
                # print(f"Name: {dienmayxanh_name}")

                dienmayxanh_images = soup.select('a.main-contain')
                if dienmayxanh_images:
                    if dienmayxanh_images[i].find('img').get('src'):
                        dienmayxanh_image = dienmayxanh_images[i].find('img')['src']
                    elif dienmayxanh_images[i].find_all('img', class_='lazyload'):
                        dienmayxanh_image = dienmayxanh_images[i].find_all('img', class_='lazyload')[0]['data-src']
                    elif dienmayxanh_images[i].find_all('img', class_='lazyloaded'):
                        dienmayxanh_image = dienmayxanh_images[i].find_all('img', class_='lazyloaded')[0]['data-src']  
                    else:
                        dienmayxanh_image = '0'
                else:
                    dienmayxanh_image = '0'
                
                try:
                    # Robust empty price check
                    if not soup.select('a.main-contain>strong.price')[i].getText().strip() or soup.select('a.main-contain>strong.price')[i].getText().strip() == ' ':
                        dienmayxanh_price = '0'
                    else:
                        dienmayxanh_price = soup.select('a.main-contain>strong.price')[i].getText().strip().upper()
                        dienmayxanh_price = dienmayxanh_price.strip('₫')
                        dienmayxanh_price = re.sub("[^0-9]", "", dienmayxanh_price)
                except:
                    continue        

                product = {
                    "name": dienmayxanh_name,
                    "price": dienmayxanh_price,
                    "image": dienmayxanh_image,
                    "link": dienmayxanh_url
                }

                # Kiểm tra xem sản phẩm nào price=0 thì không lưu vào matching_products và xoá product đó
                if product["price"] == '0':
                    continue

                matching_products.append(product)
          
        # Nếu không có sản phẩm nào thì trả về None
        if not matching_products:
            raise Exception("Không tìm thấy sản phẩm!") 
      

        dienmayxanh_price_float = [float(ele['price']) for ele in matching_products]  
        thresh = np.quantile(dienmayxanh_price_float, q = 0.05)
        
        # remome thresh 
        dienmayxanh_price = [ele['price'] for ele, price_float  in zip(matching_products, dienmayxanh_price_float) if price_float>=thresh]    
        
        # Name
        # ' '.join(element_names[0].get_text().replace("\n", "").split())
        dienmayxanh_name = [ele["name"] for ele, price_float in zip(matching_products, dienmayxanh_price_float) if price_float>=thresh]
                
        # Image
        # element_image[0].attrs['src']
        dienmayxanh_image = [ele["image"] for ele, price_float in zip(matching_products, dienmayxanh_price_float) if price_float>=thresh]
        
        # Lọc giá bé nhất
        dienmayxanh_price_float = [ele for ele in dienmayxanh_price_float if ele>=thresh]
        # min(enumerate(a), key=lambda x: x[1])[0]
        index_min = min(enumerate(dienmayxanh_price_float), key=lambda x: x[1])[0]
        
        dienmayxanh_name = dienmayxanh_name[index_min]
        dienmayxanh_price = dienmayxanh_price[index_min]
        dienmayxanh_image = dienmayxanh_image[index_min]
        
        
        print("Điện Máy Xanh:")
        print("Tên Sản Phẩm:", dienmayxanh_name)
        print("Giá:", dienmayxanh_price)
        print("Link Ảnh:", dienmayxanh_image)
        print("Link:", dienmayxanh_url)
        print("---------------------------------")
    
        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url

    except Exception as e:
        print(f"Lỗi: {e}")
        print("Điện Máy Xanh: No product found!")
        print("---------------------------------")
        dienmayxanh_price = '0'
        dienmayxanh_name = '0'
        dienmayxanh_image = '0'
        dienmayxanh_url = '0'
        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url
# ******************************************************************************************************

dienmayxanh("Macbook pro")

Lỗi: HTTPSConnectionPool(host='www.dienmayxanh.com', port=443): Max retries exceeded with url: /tim-kiem?key=Macbook+pro (Caused by SSLError(SSLError(1, '[SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] unsafe legacy renegotiation disabled (_ssl.c:1133)')))
Điện Máy Xanh: No product found!
---------------------------------


('0', '0', '0', '0')

In [18]:
import imp
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pathlib import Path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from django.conf import settings
import re

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import numpy as np

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

from fuzzywuzzy import fuzz


# Thiết lập options cho Chrome headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

# Khởi tạo driver
service = Service(ChromeDriverManager().install())
sendo_driver = webdriver.Chrome(options=chrome_options, service=service)
dienmaycholon_driver = webdriver.Chrome(options=chrome_options, service=service)
chotot_driver = webdriver.Chrome(options=chrome_options, service=service)
dienmayxanh_driver = webdriver.Chrome(options=chrome_options, service=service)

In [17]:
import imp
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pathlib import Path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from django.conf import settings
import re

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import numpy as np

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

from fuzzywuzzy import fuzz
chrome_options = Options()
chotot_driver = webdriver.Chrome(options=chrome_options, service=service)

url = "https://www.chotot.com/mua-ban?q=Iphone"
chotot_driver.get(url)
chotot_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.158)
Stacktrace:
	GetHandleVerifier [0x0073C183+48259]
	(No symbol) [0x006CCAB1]
	(No symbol) [0x005C0A17]
	(No symbol) [0x0059E02B]
	(No symbol) [0x0062742E]
	(No symbol) [0x00639476]
	(No symbol) [0x00620B36]
	(No symbol) [0x005F570D]
	(No symbol) [0x005F62CD]
	GetHandleVerifier [0x009F6613+2908435]
	GetHandleVerifier [0x00A33C2B+3159851]
	GetHandleVerifier [0x007D513B+674875]
	GetHandleVerifier [0x007DB2FC+699900]
	(No symbol) [0x006D62B4]
	(No symbol) [0x006D2308]
	(No symbol) [0x006D249C]
	(No symbol) [0x006C4C20]
	BaseThreadInitThunk [0x750F7BA9+25]
	RtlInitializeExceptionChain [0x7727BE3B+107]
	RtlClearBits [0x7727BDBF+191]


In [20]:
url = "https://www.chotot.com/mua-ban?q=Iphone"
chotot_driver.get(url)
chotot_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

soup = BeautifulSoup(chotot_driver.page_source, 'lxml')

# Tìm kiếm element
element_name = soup.select("h3.commonStyle_adTitle__g520j")
print(len(element_name))

element_price = soup.select("p.AdBody_adPriceNormal___OYFU")
print(len(element_price))

element_image = soup.select("div.ListAds_ListAds__rEu_9.col-xs-12.no-padding li.AdItem_wrapperAdItem__S6qPH.AdItem_big__70CJq picture.webpimg-container")
print(len(element_image))



25
25
5


In [36]:
elements = soup.select("div.ListAds_ListAds__rEu_9.col-xs-12.no-padding li.AdItem_wrapperAdItem__S6qPH.AdItem_big__70CJq")
chotot_name = []
chotot_price = []
chotot_image = []
if len(elements) == 0:
    raise Exception("Không tìm thấy sản phẩm!")
    
for ele in elements:
    try:
        name_element = ele.select_one("h3.commonStyle_adTitle__g520j")
        if name_element is not None:
            name_element = ' '.join(name_element.get_text().replace("\n", "").split())
    
        price_element = ele.select_one("p.AdBody_adPriceNormal___OYFU")
        if price_element is not None:
            price_element = price_element.get_text().strip(' đ').strip().replace(".", "")
        else:
            price_element = '0'
        
        image_element = ele.select_one("picture.webpimg-container img")
        if image_element is not None:
            image_element = image_element.get("src")
        else:   
            image_element = '0'
        
        
        chotot_name.append(name_element)
        chotot_price.append(price_element)
        chotot_image.append(image_element)
    except:
        pass

print(len(chotot_name))    
print(len(chotot_price))
print(len(chotot_image))


25
25
25


In [35]:
chotot_image

['https://cdn.chotot.com/E0KQKuJ4D_vJMnFSL6KD66yy_zdtJZsR7Gy_gVGw81o/preset:listing/plain/1c84163056723393ada3f893b03e0013-2877988238855219218.jpg',
 'https://cdn.chotot.com/JE6d_aSlWg1n8Rs71TgCBkDuzhvRXXmzzQFVp4BgCME/preset:listing/plain/a750ab58be03751148bd6972f90ce059-2832728539154461191.jpg',
 'https://cdn.chotot.com/L3SzdAgqE6P0_qmThtReoxmKAgkr1Mf4yAWaymc7KeQ/preset:listing/plain/ae5e8b65c174e0ec8e6b477066c1f696-2861479935712377598.jpg',
 'https://cdn.chotot.com/SDZ10_ldAdW755osTDVd2ALqrjUQ0V65mOYqyy9Q6Qg/preset:listing/plain/e234e8bc86b9f1fac9881db3f093d024-2834340461066101228.jpg',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 'https://cdn.chotot.com/jvsd4yMAmCo3g-pcJ12NmbadILpwmJy0q0Y5nU8ExDA/preset:listing/plain/8e0fa725538ae8db8f3fc72a0c1ef872-2875366104518593608.jpg']

In [1]:
import imp
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pathlib import Path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import numpy as np

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

from fuzzywuzzy import fuzz


# Thiết lập options cho Chrome headless
service = Service(ChromeDriverManager().install())
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

C:\Users\truon\AppData\Local\Temp\ipykernel_3984\2877795523.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
d:\anaconda3\envs\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [9]:
chotot_driver = webdriver.Chrome(options=chrome_options, service=service)   


def check_val_float(val):
    try:
        float(val)
        return True
    except ValueError:
        return False

# *************************Chợ Tốt*************************
def chotot(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "-")
        chotot_url = f'https://www.chotot.com/mua-ban?q={name2}'

        chotot_driver.get(chotot_url) 
        chotot_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        chotot_driver.implicitly_wait(10)  # Chờ đợi element xuất hiện 

        print("---------------------------------")
        print("\nSearching in Chợ Tốt...")
        soup = BeautifulSoup(chotot_driver.page_source, 'lxml')
        
        elements = soup.select("div.ListAds_ListAds__rEu_9.col-xs-12.no-padding li.AdItem_wrapperAdItem__S6qPH.AdItem_big__70CJq")
        chotot_name = []
        chotot_price = []
        chotot_image = []
        if len(elements) == 0:
            raise Exception("Không tìm thấy sản phẩm!")
            
        for ele in elements:
            try:
                name_element = ele.select_one("h3.commonStyle_adTitle__g520j")
                if name_element is not None:
                    name_element = name_element.get_text()
                else:
                    name_element = 'NO NAME'
                    
                price_element = ele.select_one("p.AdBody_adPriceNormal___OYFU")
                if price_element is not None:
                    price_element = price_element.get_text().strip(' đ').strip().replace(".", "")
                else:
                    price_element = '0'
                
                image_element = ele.select_one("picture.webpimg-container img")
                if image_element is not None:
                    image_element = image_element.get("src")
                else:   
                    image_element = '0'
                
                
                chotot_name.append(name_element)
                chotot_price.append(price_element)
                chotot_image.append(image_element)
            except:
                pass

        # Lấy thông tin sản phẩm   

        print(chotot_name)
        print(chotot_price)
        print(chotot_image)
        
        chotot_check_vald_product = [check_val_float(ele) for ele in chotot_price]
        if not any(chotot_check_vald_product):
            raise Exception("Không tìm thấy sản phẩm!")
        
        # Lọc ra những giá trị không hợp lệ
        
        chotot_name = [ele for ele, check in zip(chotot_name, chotot_check_vald_product) if check]
        chotot_price = [ele for ele, check in zip(chotot_price, chotot_check_vald_product) if check]    
        chotot_image = [ele for ele, check in zip(chotot_image, chotot_check_vald_product) if check]
        
        chotot_price_float = [float(ele) for ele in chotot_price]
        thresh = np.quantile(chotot_price_float, q = 0.55)
        # remome thresh 
        chotot_price = [ele for ele, price_float  in zip(chotot_price, chotot_price_float) if price_float>=thresh]    
        
        # Name
        chotot_name = [ele for ele, price_float in zip(chotot_name, chotot_price_float) if price_float>=thresh]
                
        # Image
        # element_image[0].attrs['src']
        chotot_image = [ele for ele, price_float in zip(chotot_image, chotot_price_float) if price_float>=thresh]
        
        # Lọc giá bé nhất
        chotot_price_float = [ele for ele in chotot_price_float if ele>=thresh]
        
        # Check if chotot_price_float is not empty
        if chotot_price_float:
            index_min = min(enumerate(chotot_price_float), key=lambda x: x[1])[0]   
            
            chotot_name = chotot_name[index_min]
            chotot_price = chotot_price[index_min]
            chotot_image = chotot_image[index_min]
            
            print("Chợ Tốt:")
            print("Tên Sản Phẩm:", chotot_name)
            print("Giá:", chotot_price)
            print("Link Ảnh:", chotot_image)
            print("Link:", chotot_url)
            print("---------------------------------")

            return chotot_price, chotot_name[0:50], chotot_image, chotot_url        
        else:
            print("chotot_price_float is empty")
            return '0', '0', '0', '0'

    except Exception as e:
        print(f"Lỗi: {e}")
        chotot_price = '0'
        chotot_name = '0'
        chotot_image = '0'
        chotot_url = '0'
        return chotot_price, chotot_name, chotot_image, chotot_url
    
# ******************************************

chotot("iphone")

---------------------------------

Searching in Chợ Tốt...
['iPhone 12 12 mini 13 13 mini 64GB 128GB 256GB', 'NO NAME', 'Màn hình iphone X-Xs-Xsm-Xr-11-Pro-ProMax-12-Prmax', 'màn hình zin iphone , thay pin , fix face id ,,vv', 'iPhone 11 vs XR 64GB 128GB 256GB BH 1 đổi 1', 'NO NAME', '🔥IPHONE 11 64G /128G BẢO HÀNH 24 THÁNG', 'Iphone 12prm', 'NO NAME', 'IPhone 5se', 'iPhone 6', 'iPhone Xs', 'NO NAME', 'iphone 5s', 'iphone 11', 'Iphone 12promax', 'iphone 11', 'iphone 3promax', 'Iphone 6s', 'iphone 6s', 'Iphone 6Plus', 'iphone 14pro', 'Iphone 12pro', 'iphone 5c', 'Iphone 5', 'iPhone 8', 'Iphone 14prm', 'IPHONE XSM', 'iPhone xs']
['4999000', '0', '520000', '500000', '4100000', '0', '3950000', '11500000', '0', '800000', '300000', '3500000', '0', '750000', '3900000', '12700000', '5600000', '800000', '750000', '1000000', '890000', '15900000', '7300000', '450000', '350000', '1600000', '19000000', '6800000', '3700000']
['https://cdn.chotot.com/EzMQRuh1b3fMPzHFyR-pYut-QDhvy3qaseFFPGx2rg8/preset:

('3500000', 'iPhone Xs', '0', 'https://www.chotot.com/mua-ban?q=iphone')

In [2]:
url = "https://www.chotot.com/mua-ban?q=iphone+15+pro+max"
chotot_driver.get(url)

In [3]:
soup = BeautifulSoup(chotot_driver.page_source, 'lxml')
soup

<html lang="vi"><head><link as="style" href="https://static.chotot.com/storage/APP_WRAPPER/v4/4.20.0/header.css" rel="preload"/><link as="style" href="https://static.chotot.com/storage/APP_WRAPPER/v4/4.20.0/footer.css" rel="preload"/><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><link href="https://static.chotot.com/storage/APP_WRAPPER/v4/4.20.0/header.css" rel="stylesheet"/><link href="https://static.chotot.com/storage/APP_WRAPPER/v4/4.20.0/footer.css" rel="stylesheet"/><title>Mua Bán Rao Vặt Nhanh Chóng, Uy Tín Tại Chợ Tốt Toàn quốc</title><meta content="Mua Bán Rao Vặt Nhanh Chóng, Uy Tín Tại Chợ Tốt Toàn quốc" property="og:title"/><meta content="https://static.chotot.com/storage/marketplace/ct_orange_c2c_200.jpg" property="og:image"/><meta content="https://www.chotot.com/mua-ban?q=iphone%252015%2520pro%2520max" property="og:url"/><meta content="website" property="og:type"/><meta content="Chợ Tốt - Website mua bán rao vặt của người Việt với hàng ngàn món

In [4]:
element_names = soup.select("h3.commonStyle_adTitle__g520j")
print(element_names)

[<h3 class="commonStyle_adTitle__g520j">iPhone 15 Pro Max 256G Ti Tan ViệtNam 100% BH 12T</h3>, <h3 class="commonStyle_adTitle__g520j">iphone 14 Pro vs 14 Pro Max 15 pro max 256GB</h3>, <h3 class="commonStyle_adTitle__g520j">✅IPHONE 11PRM256G QUỐC TẾ GÓP  0D CHỈ CẦN CMND</h3>, <h3 class="commonStyle_adTitle__g520j">🔥IPHONE 12 PRO MAX QTẾ GOLD TRẢ GÓP 18T BAO DUYỆT</h3>, <h3 class="commonStyle_adTitle__g520j">iPhone 15 Pro Max 256GB Quốc tế</h3>, <h3 class="commonStyle_adTitle__g520j">iPhone 15 pro max 256 trắng new seal</h3>, <h3 class="commonStyle_adTitle__g520j">iphone 15 pro max titan tự nhiên 256gb</h3>, <h3 class="commonStyle_adTitle__g520j">Iphone 15 Pro Max</h3>, <h3 class="commonStyle_adTitle__g520j">iphone 15 Pro Max Titan tự nhiên fullbox new</h3>, <h3 class="commonStyle_adTitle__g520j">iPhone 15 Pro Max 256GB Quốc tế mất Face ID</h3>, <h3 class="commonStyle_adTitle__g520j">IPHONE 15 PRO MAX NEW 100% CHƯA ACTIVE - BH 12TH</h3>, <h3 class="commonStyle_adTitle__g520j">Iphone 15

In [5]:
print(' '.join(element_names[0].get_text().replace("\n", "").split()))

iPhone 15 Pro Max 256G Ti Tan ViệtNam 100% BH 12T


In [6]:
element_price = soup.select("p.AdBody_adPriceNormal___OYFU")
print(element_price)

[<p class="AdBody_adPriceNormal___OYFU">28.990.000 đ </p>, <p class="AdBody_adPriceNormal___OYFU">16.999.000 đ </p>, <p class="AdBody_adPriceNormal___OYFU">5.000.000 đ </p>, <p class="AdBody_adPriceNormal___OYFU">8.150.000 đ </p>, <p class="AdBody_adPriceNormal___OYFU">28.400.000 đ </p>, <p class="AdBody_adPriceNormal___OYFU">29.500.000 đ<!-- --> </p>, <p class="AdBody_adPriceNormal___OYFU">25.500.000 đ<!-- --> </p>, <p class="AdBody_adPriceNormal___OYFU">27.200.000 đ<!-- --> </p>, <p class="AdBody_adPriceNormal___OYFU">27.900.000 đ<!-- --> </p>, <p class="AdBody_adPriceNormal___OYFU">17.300.000 đ<!-- --> </p>, <p class="AdBody_adPriceNormal___OYFU">26.290.000 đ<!-- --> </p>, <p class="AdBody_adPriceNormal___OYFU">1.600.000 đ<!-- --> </p>, <p class="AdBody_adPriceNormal___OYFU">20.900.000 đ<!-- --> </p>, <p class="AdBody_adPriceNormal___OYFU">26.500.000 đ<!-- --> </p>, <p class="AdBody_adPriceNormal___OYFU">25.000.000 đ<!-- --> </p>, <p class="AdBody_adPriceNormal___OYFU">26.800.000 đ<

In [7]:
import numpy as np

In [8]:
# Giá

chotot_price = [ele.get_text().strip(' đ').strip().replace(".", "") for ele in element_price]
chotot_price_float = [float(ele) for ele in chotot_price]
thresh = np.quantile(chotot_price_float, q = 0.25)
# remome thresh 
chotot_price = [ele for ele, price_float  in zip(chotot_price, chotot_price_float) if price_float>=thresh]

print(chotot_price)

['28990000', '28400000', '29500000', '25500000', '27200000', '27900000', '26290000', '20900000', '26500000', '25000000', '26800000', '26900000', '28800000', '23300000', '20000000', '25990000', '18999999', '24000000']


In [10]:
# ['29990000', '29490000', '29990000', '29690000', '35990000', '35490000', '42690000', '35490000', '35990000', '42690000', '29990000', '42690000', '32990000', '42690000', '24890000', '24890000', '27590000', '23990000', '27590000', '24890000', '27590000', '27590000', '34490000', '34490000', '34490000', '39990000', '34490000', '309000', '189000', '189000', '259000', '199000', '194000', '449000', '150000', '220000', '150000', '150000', '180000', '280000', '179000', '289000', '199000']

# Lọc ra giá trị ngoại lai
# np.quantile(chotot_price, q = 0.25)

In [11]:
print(element_price[0].get_text().strip('đ').split(":")[-1])

28.990.000 đ 


In [12]:
element_image = soup.select("picture.webpimg-container img")
print(element_image)

[<img alt="iPhone 15 Pro Max 256G Ti Tan ViệtNam 100% BH 12T" class="AdThumbnail_thumbnailDefault__mF_DR priority" height="128" loading="eager" src="https://cdn.chotot.com/Xu1vJysVOqZ_FKv3YCRrBis5DA40NOQ4XzeH9Mol7Fw/preset:listing/plain/c7161c8ca0a665000e40e00e6aa4a7ea-2845345700940301203.jpg" title="iPhone 15 Pro Max 256G Ti Tan ViệtNam 100% BH 12T" width="128"/>, <img alt="iphone 14 Pro vs 14 Pro Max 15 pro max 256GB" class="AdThumbnail_thumbnailDefault__mF_DR priority" height="128" loading="eager" src="https://cdn.chotot.com/n9nFpylfwYiOoDc5EFnFAZZ81y3lFSPPKZHRU9pxeOY/preset:listing/plain/3b822921e5e9048af8b343afab32f889-2855088174207725146.jpg" title="iphone 14 Pro vs 14 Pro Max 15 pro max 256GB" width="128"/>, <img alt="✅IPHONE 11PRM256G QUỐC TẾ GÓP  0D CHỈ CẦN CMND" class="AdThumbnail_thumbnailDefault__mF_DR priority" height="128" loading="eager" src="https://cdn.chotot.com/Mbxo2Fgbsd5_oPBw3HbFH5beFJTVaN_z1iSiYZ5NY-c/preset:listing/plain/2152f045f643d31e277fd4f15f616d5f-287394943

In [13]:
element_image[0].attrs['src'] or element_image[0].attrs['data-src']

'https://cdn.chotot.com/Xu1vJysVOqZ_FKv3YCRrBis5DA40NOQ4XzeH9Mol7Fw/preset:listing/plain/c7161c8ca0a665000e40e00e6aa4a7ea-2845345700940301203.jpg'

In [26]:
import numpy as np

In [16]:
dienmayxanh_driver = webdriver.Chrome(options=chrome_options, service=service)

In [ ]:
# ****************************************Điện Máy Xanh*****************************************
def dienmayxanh(name):
    try:
        name2 = name.replace(" ", "+")
        dienmayxanh_url = f'https://www.dienmayxanh.com/tim-kiem?key={name2}'

        res = requests.get(dienmayxanh_url, headers=headers)
        print("\nSearching in Điện Máy Xanh...")
        soup = BeautifulSoup(res.text, 'html.parser')

        dienmayxanh_page = soup.select('a.main-contain')
        dienmayxanh_page_length = int(len(dienmayxanh_page))        
        
        matching_products = []  # List to store matching products
        for i in range(0, dienmayxanh_page_length):
            dienmayxanh_name = soup.select('a.main-contain>h3')[i].getText().strip().upper()
   
            similarity_score = fuzz.ratio(name.upper(), dienmayxanh_name)
            # print(f"Similarity Score: {similarity_score}")
                
            if similarity_score >= 10:
                dienmayxanh_name = soup.select('a.main-contain>h3')[i].getText().strip()
                # print(f"Name: {dienmayxanh_name}")

                dienmayxanh_images = soup.select('a.main-contain')
                if dienmayxanh_images:
                    if dienmayxanh_images[i].find('img').get('src'):
                        dienmayxanh_image = dienmayxanh_images[i].find('img')['src']
                    elif dienmayxanh_images[i].find_all('img', class_='lazyload'):
                        dienmayxanh_image = dienmayxanh_images[i].find_all('img', class_='lazyload')[0]['data-src']
                    elif dienmayxanh_images[i].find_all('img', class_='lazyloaded'):
                        dienmayxanh_image = dienmayxanh_images[i].find_all('img', class_='lazyloaded')[0]['data-src']  
                    else:
                        dienmayxanh_image = '0'
                else:
                    dienmayxanh_image = '0'
                
                try:
                    # Robust empty price check
                    if not soup.select('a.main-contain>strong.price')[i].getText().strip() or soup.select('a.main-contain>strong.price')[i].getText().strip() == ' ':
                        dienmayxanh_price = '0'
                    else:
                        dienmayxanh_price = soup.select('a.main-contain>strong.price')[i].getText().strip().upper()
                        dienmayxanh_price = dienmayxanh_price.strip('₫')
                        dienmayxanh_price = re.sub("[^0-9]", "", dienmayxanh_price)
                except:
                    continue        

                product = {
                    "name": dienmayxanh_name,
                    "price": dienmayxanh_price,
                    "image": dienmayxanh_image,
                    "link": dienmayxanh_url
                }


        
       

      
        
    
        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url

    except Exception as e:
        print(f"Lỗi: {e}")
        print("Điện Máy Xanh: No product found!")
        print("---------------------------------")
        dienmayxanh_price = '0'
        dienmayxanh_name = '0'
        dienmayxanh_image = '0'
        dienmayxanh_url = '0'
        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url
# ******************************************************************************************************

In [17]:
# *************************Điện máy xanh*************************
def dienmayxanh(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "-")
        dienmayxanh_url = f"https://www.dienmayxanh.com/tim-kiem?key={name2}"

        dienmayxanh_driver.get(dienmayxanh_url) 
        print("---------------------------------")
        print("\nSearching in Điện Máy Xanh...")
        soup = BeautifulSoup(dienmayxanh_driver.page_source, 'lxml')
        
        # Tìm kiếm element
        element_name = soup.select("a.main-contain>h3")

        element_price = soup.select("a.main-contain>strong.price")
        
        element_image = soup.select("a.main-contain > div > img")

        # Lấy thông tin sản phẩm   
               
        # Giá
        # re.sub("[^0-9]", ""
        dienmayxanh_price = [re.sub("[^0-9]", "", ele.get_text().strip('₫').strip().replace(".", "")) for ele in element_price]
        dienmayxanh_price_float = [float(ele) for ele in dienmayxanh_price]
        thresh = np.quantile(dienmayxanh_price_float, q = 0.25)
        # remome thresh 
        dienmayxanh_price = [ele for ele, price_float  in zip(dienmayxanh_price, dienmayxanh_price_float) if price_float>=thresh]    
        
        # Name
        # ' '.join(element_names[0].get_text().replace("\n", "").split())
        dienmayxanh_name = [' '.join(ele.get_text().replace("\n", "").split()) for ele, price_float in zip(element_name, dienmayxanh_price_float) if price_float>=thresh]
                
        # Image
        dienmayxanh_image = [ele.attrs['data-src'] for ele, price_float in zip(element_image, dienmayxanh_price_float) if price_float>=thresh]
        if not dienmayxanh_image:
            dienmayxanh_image = [ele.attrs['src'] for ele, price_float in zip(element_image, dienmayxanh_price_float) if price_float>=thresh]
        
        # Lọc giá bé nhất
        dienmayxanh_price_float = [ele for ele in dienmayxanh_price_float if ele>=thresh]
        # min(enumerate(a), key=lambda x: x[1])[0]
        index_min = min(enumerate(dienmayxanh_price_float), key=lambda x: x[1])[0]
        
        dienmayxanh_name = dienmayxanh_name[index_min]
        dienmayxanh_price = dienmayxanh_price[index_min]
        dienmayxanh_image = dienmayxanh_image[index_min]
        
        print("Điện Máy Xanh:")
        print("Tên Sản Phẩm:", dienmayxanh_name)
        print("Giá:", dienmayxanh_price)
        print("Link Ảnh:", dienmayxanh_image)
        print("Link:", dienmayxanh_url)
        print("---------------------------------")

        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url

    except Exception as e:
        print(f"Lỗi: {e}")
        dienmayxanh_price = '0'
        dienmayxanh_name = '0'
        dienmayxanh_image = '0'
        dienmayxanh_url = '0'
        return dienmayxanh_price, dienmayxanh_name, dienmayxanh_image, dienmayxanh_url
    
# ******************************************

dienmayxanh("Macbook pro")

---------------------------------

Searching in Điện Máy Xanh...
Điện Máy Xanh:
Tên Sản Phẩm: Bao da Macbook Pro 13 inch Apple
Giá: 3390000
Link Ảnh: https://cdn.tgdd.vn/Products/Images/1363/313159/bo-dan-macbook-pro-2020-m1-m2-13-inch-6-in-1-innostyle-iscs2338-space-gray-thumb-600x600.jpg
Link: https://www.dienmayxanh.com/tim-kiem?key=Macbook-pro
---------------------------------


('3390000',
 'Bao da Macbook Pro 13 inch Apple',
 'https://cdn.tgdd.vn/Products/Images/1363/313159/bo-dan-macbook-pro-2020-m1-m2-13-inch-6-in-1-innostyle-iscs2338-space-gray-thumb-600x600.jpg',
 'https://www.dienmayxanh.com/tim-kiem?key=Macbook-pro')

In [23]:
chotot_driver = webdriver.Chrome(options=chrome_options, service=service)

In [24]:
#********************************************Chợ tốt**********************************************************************

def chotot(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "+")
        chotot_url = f'https://www.chotot.com/mua-ban?q={name2}'

        chotot_driver.get(chotot_url)

        # Tìm kiếm danh sách element sản phẩm
        product_elements = chotot_driver.find_elements(By.CSS_SELECTOR, "div.ListAds_ListAds__rEu_9")

        products = []
        for product_element in product_elements:
            # Lấy thông tin sản phẩm
            try:
                name_element = product_element.find_element(By.CSS_SELECTOR, "h3.commonStyle_adTitle__g520j")
                price_element = product_element.find_element(By.CSS_SELECTOR, "p.AdBody_adPriceNormal___OYFU")
                image_element = product_element.find_element(By.CSS_SELECTOR, "picture.webpimg-container img")

                chotot_name = name_element.text
                chotot_price = price_element.text.strip('đ')
                chotot_image = image_element.get_attribute("src")

                # Lưu thông tin vào dictionary
                product = {
                    "name": chotot_name,
                    "price": chotot_price,
                    "image": chotot_image,
                    "link": chotot_url
                }
                products.append(product)
            except NoSuchElementException:
                # Bỏ qua nếu element không tồn tại
                pass

        # In thông tin sản phẩm (nếu có)
        if products:
            for product in products:
                print("Chợ tốt:")
                print("Tên Sản Phẩm:", product["name"])
                print("Giá:", product["price"])
                print("Link Ảnh:", product["image"])
                print("Link:", product["link"])
                print("---------------------------------")
        else:
            print("chotot: No product found!")
            print("---------------------------------")

        return products

    except Exception as e:
        print(f"Lỗi: {e}")
        return []
# *********************************************************************************************************

chotot("Macbook pro")

Chợ tốt:
Tên Sản Phẩm: MacBook Pro 14inch M3 pro / 18G / 1TB Fullbox
Giá: 39.900.000 
Link Ảnh: https://cdn.chotot.com/U3RPgfORhli68NdX2TJ1jzRf-ER4Kk_KF_0nvK-gz3E/preset:listing/plain/04a08e109ed383b1ec8b30f983d021f6-2874943880363026443.jpg
Link: https://www.chotot.com/mua-ban?q=Macbook+pro
---------------------------------


[{'name': 'MacBook Pro 14inch M3 pro / 18G / 1TB Fullbox',
  'price': '39.900.000 ',
  'image': 'https://cdn.chotot.com/U3RPgfORhli68NdX2TJ1jzRf-ER4Kk_KF_0nvK-gz3E/preset:listing/plain/04a08e109ed383b1ec8b30f983d021f6-2874943880363026443.jpg',
  'link': 'https://www.chotot.com/mua-ban?q=Macbook+pro'}]

In [20]:
import imp
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pathlib import Path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from django.conf import settings
import re

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import numpy as np

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

from fuzzywuzzy import fuzz


# Thiết lập options cho Chrome headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

# Khởi tạo driver
service = Service(ChromeDriverManager().install())
chotot_driver = webdriver.Chrome(options=chrome_options, service=service)

In [9]:
def sendo(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "+")
        sendo_url = f"https://www.sendo.vn/tim-kiem?q={name2}"
        print("Searching in Sendo...")
        sendo_driver.get(sendo_url)
        
        
        # Tìm kiếm danh sách element
        name_elements = WebDriverWait(sendo_driver, 10).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.d7ed-Vp2Ugh._0032-Zwkt7j"))
                         )
        # Tìm kiếm danh sách element giá sản phẩm
        price_elements = WebDriverWait(sendo_driver, 10).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span._0032-GpBMYp._0032-npoTU_.d7ed-CLUDGW.d7ed-AHa8cD.d7ed-giDKVr"))
                        )
        # Tìm kiếm danh sách element ảnh sản phẩm
        image_elements = WebDriverWait(sendo_driver, 10).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.d7ed-a1ulZz img"))
                        )

        # # Kiểm tra số lượng element
        num_elements = len(name_elements)
        # print(num_elements)
        if num_elements != len(price_elements) or num_elements != len(image_elements):
            print("Sendo: Số lượng element không khớp!")
            return []

        # Lấy danh sách sản phẩm
        products = []

        for i in range(num_elements):
            # Lấy tên sản phẩm
            sendo_name = name_elements[i].text

            # Lấy giá sản phẩm
            sendo_price = price_elements[i].text.strip('đ')
            # '900000đ-999000 xử lý chỉ lấy giá ở sau dấu "-"
            sendo_price = sendo_price.split("-")[-1]
            sendo_price = sendo_price.replace(".", "")
                            

            # Lấy link ảnh sản phẩm
            image_src = image_elements[i].get_attribute("data-src")
            if not image_src:
                image_src = image_elements[i].get_attribute("src")
            
            # Lưu thông tin vào dictionary
            product = {
                "name": sendo_name,
                "price": sendo_price,
                "image": image_src,
                "link": sendo_url
            }

            products.append(product)

        sendo_price_float = [float(ele["price"]) for ele in products]
        thresh = np.quantile(sendo_price_float, q = 0.55)
        # remome thresh
        products = [ele for ele in products if float(ele["price"])>=thresh]
        
        # Lọc giá bé nhất
        sendo_price_float = [float(ele["price"]) for ele in products]
        # min(enumerate(a), key=lambda x: x[1])[0]
        index_min = min(enumerate(sendo_price_float), key=lambda x: x[1])[0]

        sendo_name = products[index_min]["name"]
        sendo_price = products[index_min]["price"]
        sendo_image = products[index_min]["image"]

        print("Sendo:")
        print("Tên Sản Phẩm:", sendo_name)
        print("Giá:", sendo_price)
        print("Link Ảnh:", sendo_image)
        print("Link:", sendo_url)
        print("---------------------------------")
        
        return sendo_price, sendo_name[0:50], sendo_image, sendo_url
        
                                         
    except Exception as e:
        print(f"Lỗi: {e}")
        sendo_price = '0'
        sendo_name = '0'
        sendo_link = '0'
        sendo_image = '0'
        return sendo_price, sendo_name, sendo_image, sendo_link
    
# **************************************************************************

sendo("Iphone")

Searching in Sendo...
Sendo:
Tên Sản Phẩm: iPhone 15 Plus 128GB - Xanh lá
Giá: 22390000
Link Ảnh: https://media3.scdn.vn/img4/2023/09_27/43Z3ZdImi6GWPKcloiRL_simg_de2fe0_250x250_maxb.jpg
Link: https://www.sendo.vn/tim-kiem?q=Iphone
---------------------------------


('22390000',
 'iPhone 15 Plus 128GB - Xanh lá',
 'https://media3.scdn.vn/img4/2023/09_27/43Z3ZdImi6GWPKcloiRL_simg_de2fe0_250x250_maxb.jpg',
 'https://www.sendo.vn/tim-kiem?q=Iphone')

In [28]:
import imp
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pathlib import Path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from django.conf import settings
import re

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import numpy as np

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

from fuzzywuzzy import fuzz

# ****************************************Điện Máy Xanh*****************************************
def dienmayxanh(name):
    try:
        name2 = name.replace(" ", "+")
        dienmayxanh_url = f'https://www.dienmayxanh.com/tim-kiem?key={name2}'

        res = requests.get(dienmayxanh_url, headers=headers)
        print("\nSearching in Điện Máy Xanh...")
        soup = BeautifulSoup(res.text, 'html.parser')

        dienmayxanh_page = soup.select('a.main-contain')
        dienmayxanh_page_length = int(len(dienmayxanh_page))        
        
        matching_products = []  # List to store matching products
        for i in range(0, dienmayxanh_page_length):
            dienmayxanh_name = soup.select('a.main-contain>h3')[i].getText().strip().upper()
   
            similarity_score = fuzz.ratio(name.upper(), dienmayxanh_name)
            # print(f"Similarity Score: {similarity_score}")
                
            if similarity_score >= 10:
                dienmayxanh_name = soup.select('a.main-contain>h3')[i].getText().strip()
                # print(f"Name: {dienmayxanh_name}")

                dienmayxanh_images = soup.select('a.main-contain')
                if dienmayxanh_images:
                    if dienmayxanh_images[i].find('img').get('src'):
                        dienmayxanh_image = dienmayxanh_images[i].find('img')['src']
                    elif dienmayxanh_images[i].find_all('img', class_='lazyload'):
                        dienmayxanh_image = dienmayxanh_images[i].find_all('img', class_='lazyload')[0]['data-src']
                    elif dienmayxanh_images[i].find_all('img', class_='lazyloaded'):
                        dienmayxanh_image = dienmayxanh_images[i].find_all('img', class_='lazyloaded')[0]['data-src']  
                    else:
                        dienmayxanh_image = '0'
                else:
                    dienmayxanh_image = '0'
                
                try:
                    # Robust empty price check
                    if not soup.select('a.main-contain>strong.price')[i].getText().strip() or soup.select('a.main-contain>strong.price')[i].getText().strip() == ' ':
                        dienmayxanh_price = '0'
                    else:
                        dienmayxanh_price = soup.select('a.main-contain>strong.price')[i].getText().strip().upper()
                        dienmayxanh_price = dienmayxanh_price.strip('₫')
                        dienmayxanh_price = re.sub("[^0-9]", "", dienmayxanh_price)
                except:
                    continue        

                product = {
                    "name": dienmayxanh_name,
                    "price": dienmayxanh_price,
                    "image": dienmayxanh_image,
                    "link": dienmayxanh_url
                }

                # Kiểm tra xem sản phẩm nào price=0 thì không lưu vào matching_products và xoá product đó
                if product["price"] == '0':
                    del product
                else:
                    matching_products.append(product)
          
        # Nếu không có sản phẩm nào thì trả về None
        if not matching_products:
            return None  
        
        #------------------------------------------------------------------------
        # print("Matching Products:")
        # for product in matching_products:
        #     print("Tên Sản Phẩm:", product["name"])
        #     print("Giá:", product["price"])
        #     print("Link Ảnh:", product["image"])
        #     print("Link:", product["link"])
        #     print("---------------------------------")
        
        # # Find the cheapest product
        # cheapest_product = min(matching_products, key=lambda p: int(p["price"].replace('.', '')))
                
        # if cheapest_product:
        #     print("Tên Sản Phẩm:", cheapest_product["name"])
        #     print("Giá:", cheapest_product["price"])
        #     print("Link Ảnh:", cheapest_product["image"])
        #     print("Link:", cheapest_product["link"])
        #     print("---------------------------------") 
        # else:
        #     print("No matching product found.")    
        
        # Lưu cheapest_product vào list_product
        # list_product = []
        # list_product.append(cheapest_product)
        #------------------------------------------------------------------------
      
        dienmayxanh_price = matching_products[0]["price"]
        dienmayxanh_name= matching_products[0]["name"]
        dienmayxanh_image= matching_products[0]["image"]
        dienmayxanh_url= matching_products[0]["link"]
        print("Điện Máy Xanh:")
        print("Tên Sản Phẩm:", dienmayxanh_name)
        print("Giá:", dienmayxanh_price)
        print("Link Ảnh:", dienmayxanh_image)
        print("Link:", dienmayxanh_url)
        print("---------------------------------")
    
        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url

    except Exception as e:
        print(f"Lỗi: {e}")
        print("Điện Máy Xanh: No product found!")
        print("---------------------------------")
        dienmayxanh_price = '0'
        dienmayxanh_name = '0'
        dienmayxanh_image = '0'
        dienmayxanh_url = '0'
        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url
# ******************************************************************************************************

dienmayxanh("Macbook pro")

Lỗi: HTTPSConnectionPool(host='www.dienmayxanh.com', port=443): Max retries exceeded with url: /tim-kiem?key=Macbook+pro (Caused by SSLError(SSLError(1, '[SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] unsafe legacy renegotiation disabled (_ssl.c:1133)')))
Điện Máy Xanh: No product found!
---------------------------------


('0', '0', '0', '0')

In [1]:
import imp
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pathlib import Path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from django.conf import settings
import re

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import numpy as np

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

from fuzzywuzzy import fuzz


# Thiết lập options cho Chrome headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

# Khởi tạo driver
service = Service(ChromeDriverManager().install())
sendo_driver = webdriver.Chrome(options=chrome_options, service=service)
dienmaycholon_driver = webdriver.Chrome(options=chrome_options, service=service)
chotot_driver = webdriver.Chrome(options=chrome_options, service=service)
dienmayxanh_driver = webdriver.Chrome(options=chrome_options, service=service)
# ****************************************Format**********************************************

from deep_translator import GoogleTranslator
from selenium.common.exceptions import NoSuchElementException

def translator(text):
    """
    Dịch văn bản từ ngôn ngữ hiện tại sang tiếng Anh.
    # Đối với các trang quốc tế: Amazon, ...
    """
    
    type_trans = GoogleTranslator(source='auto', target='en')
    trans = type_trans.translate(text)
    return trans

def no_accent_vietnamese(s):
    s = s.lower()
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('đ', 'd', s)
    return s

def convert(a):
    b = a.replace(" ", '')
    c = b.replace("INR", '')
    d = c.replace(",", '')
    d = d.replace("`", '')
    f = d.replace("₹", '')
    f1 = f.replace(".", '')
    f2 = f1.replace("₫", '')
    g = int(float(f2))
    return g

def format_price(price):
    """Formats a price number into a string with commas, currency symbol, and handles potential decimals.

  Args:
      price: The price as a number (float or int).

  Returns:
      A string representing the formatted price.
  """
    # Handle potential decimals
    if isinstance(price, float):
        price_str = "{:,.2f}".format(price)  # Two decimal places for floats
    else:
        price_str = "{:,}".format(price)  # No decimal places for integers

    return price_str + "đ"  # Add currency symbol (đ)
# **********************************************************************************************


# # ****************************************Điện Máy Xanh*****************************************
# def dienmayxanh(name):
#     try:
#         name2 = name.replace(" ", "+")
#         dienmayxanh_url = f'https://www.dienmayxanh.com/tim-kiem?key={name2}'

#         res = requests.get(dienmayxanh_url, headers=headers)
#         print("\nSearching in Điện Máy Xanh...")
#         soup = BeautifulSoup(res.text, 'html.parser')

#         dienmayxanh_page = soup.select('a.main-contain')
#         dienmayxanh_page_length = int(len(dienmayxanh_page))        
        
#         matching_products = []  # List to store matching products
#         for i in range(0, dienmayxanh_page_length):
#             dienmayxanh_name = soup.select('a.main-contain>h3')[i].getText().strip().upper()
   
#             similarity_score = fuzz.ratio(name.upper(), dienmayxanh_name)
#             # print(f"Similarity Score: {similarity_score}")
                
#             if similarity_score >= 10:
#                 dienmayxanh_name = soup.select('a.main-contain>h3')[i].getText().strip()
#                 # print(f"Name: {dienmayxanh_name}")

#                 dienmayxanh_images = soup.select('a.main-contain')
#                 if dienmayxanh_images:
#                     if dienmayxanh_images[i].find('img').get('src'):
#                         dienmayxanh_image = dienmayxanh_images[i].find('img')['src']
#                     elif dienmayxanh_images[i].find_all('img', class_='lazyload'):
#                         dienmayxanh_image = dienmayxanh_images[i].find_all('img', class_='lazyload')[0]['data-src']
#                     elif dienmayxanh_images[i].find_all('img', class_='lazyloaded'):
#                         dienmayxanh_image = dienmayxanh_images[i].find_all('img', class_='lazyloaded')[0]['data-src']  
#                     else:
#                         dienmayxanh_image = '0'
#                 else:
#                     dienmayxanh_image = '0'
                
#                 try:
#                     # Robust empty price check
#                     if not soup.select('a.main-contain>strong.price')[i].getText().strip() or soup.select('a.main-contain>strong.price')[i].getText().strip() == ' ':
#                         dienmayxanh_price = '0'
#                     else:
#                         dienmayxanh_price = soup.select('a.main-contain>strong.price')[i].getText().strip().upper()
#                         dienmayxanh_price = dienmayxanh_price.strip('₫')
#                         dienmayxanh_price = re.sub("[^0-9]", "", dienmayxanh_price)
#                 except:
#                     continue        

#                 product = {
#                     "name": dienmayxanh_name,
#                     "price": dienmayxanh_price,
#                     "image": dienmayxanh_image,
#                     "link": dienmayxanh_url
#                 }

#                 # Kiểm tra xem sản phẩm nào price=0 thì không lưu vào matching_products và xoá product đó
#                 if product["price"] == '0':
#                     del product
#                 else:
#                     matching_products.append(product)
          
#         # Nếu không có sản phẩm nào thì trả về None
#         if not matching_products:
#             return None  
        
#         #------------------------------------------------------------------------
#         # print("Matching Products:")
#         # for product in matching_products:
#         #     print("Tên Sản Phẩm:", product["name"])
#         #     print("Giá:", product["price"])
#         #     print("Link Ảnh:", product["image"])
#         #     print("Link:", product["link"])
#         #     print("---------------------------------")
        
#         # # Find the cheapest product
#         # cheapest_product = min(matching_products, key=lambda p: int(p["price"].replace('.', '')))
                
#         # if cheapest_product:
#         #     print("Tên Sản Phẩm:", cheapest_product["name"])
#         #     print("Giá:", cheapest_product["price"])
#         #     print("Link Ảnh:", cheapest_product["image"])
#         #     print("Link:", cheapest_product["link"])
#         #     print("---------------------------------") 
#         # else:
#         #     print("No matching product found.")    
        
#         # Lưu cheapest_product vào list_product
#         # list_product = []
#         # list_product.append(cheapest_product)
#         #------------------------------------------------------------------------
      
#         dienmayxanh_price = matching_products[0]["price"]
#         dienmayxanh_name= matching_products[0]["name"]
#         dienmayxanh_image= matching_products[0]["image"]
#         dienmayxanh_url= matching_products[0]["link"]
#         print("Điện Máy Xanh:")
#         print("Tên Sản Phẩm:", dienmayxanh_name)
#         print("Giá:", dienmayxanh_price)
#         print("Link Ảnh:", dienmayxanh_image)
#         print("Link:", dienmayxanh_url)
#         print("---------------------------------")
    
#         return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url

#     except Exception as e:
#         print(f"Lỗi: {e}")
#         print("Điện Máy Xanh: No product found!")
#         print("---------------------------------")
#         dienmayxanh_price = '0'
#         dienmayxanh_name = '0'
#         dienmayxanh_image = '0'
#         dienmayxanh_url = '0'
#         return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url
# # ******************************************************************************************************

# *************************Điện máy xanh*************************
def dienmayxanh(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "-")
        dienmayxanh_url = f"https://www.dienmayxanh.com/tim-kiem?key={name2}"

        dienmayxanh_driver.get(dienmayxanh_url) 
        print("---------------------------------")
        print("\nSearching in Điện Máy Xanh...")
        soup = BeautifulSoup(dienmayxanh_driver.page_source, 'lxml')
        
        # Tìm kiếm element
        element_name = soup.select("a.main-contain>h3")

        element_price = soup.select("a.main-contain>strong.price")
        
        element_image = soup.select("a.main-contain > div > img")

        # Lấy thông tin sản phẩm   
               
        # Giá
        # re.sub("[^0-9]", ""
        dienmayxanh_price = [re.sub("[^0-9]", "", ele.get_text().strip('₫').strip().replace(".", "")) for ele in element_price]
        dienmayxanh_price_float = [float(ele) for ele in dienmayxanh_price]
        thresh = np.quantile(dienmayxanh_price_float, q = 0.25)
        # remome thresh 
        dienmayxanh_price = [ele for ele, price_float  in zip(dienmayxanh_price, dienmayxanh_price_float) if price_float>=thresh]    
        
        # Name
        # ' '.join(element_names[0].get_text().replace("\n", "").split())
        dienmayxanh_name = [' '.join(ele.get_text().replace("\n", "").split()) for ele, price_float in zip(element_name, dienmayxanh_price_float) if price_float>=thresh]
                
        # Image
        dienmayxanh_image = [ele.attrs['data-src'] for ele, price_float in zip(element_image, dienmayxanh_price_float) if price_float>=thresh]
        if not dienmayxanh_image:
            dienmayxanh_image = [ele.attrs['src'] for ele, price_float in zip(element_image, dienmayxanh_price_float) if price_float>=thresh]
        
        # Lọc giá bé nhất
        dienmayxanh_price_float = [ele for ele in dienmayxanh_price_float if ele>=thresh]
        # min(enumerate(a), key=lambda x: x[1])[0]
        index_min = min(enumerate(dienmayxanh_price_float), key=lambda x: x[1])[0]
        
        dienmayxanh_name = dienmayxanh_name[index_min]
        dienmayxanh_price = dienmayxanh_price[index_min]
        dienmayxanh_image = dienmayxanh_image[index_min]
        
        print("Điện Máy Xanh:")
        print("Tên Sản Phẩm:", dienmayxanh_name)
        print("Giá:", dienmayxanh_price)
        print("Link Ảnh:", dienmayxanh_image)
        print("Link:", dienmayxanh_url)
        print("---------------------------------")

        return dienmayxanh_price, dienmayxanh_name[0:50], dienmayxanh_image, dienmayxanh_url

    except Exception as e:
        print(f"Lỗi: {e}")
        dienmayxanh_price = '0'
        dienmayxanh_name = '0'
        dienmayxanh_image = '0'
        dienmayxanh_url = '0'
        return dienmayxanh_price, dienmayxanh_name, dienmayxanh_image, dienmayxanh_url
    
# ******************************************


#  ****************************************Amazon**********************************************
def amazon(name):
    try:
        global amazon
        name = translator(name)
        name1 = name.replace(" ", "-")
        name2 = name.replace(" ", "+")
        amazon = f'https://www.amazon.in/{name1}/s?k={name2}'
        amazon_link = amazon
        print(amazon_link)
        res = requests.get(
            f'https://www.amazon.in/{name1}/s?k={name2}', headers=headers)
        print("\nSearching in amazon...")
        soup = BeautifulSoup(res.text, 'html.parser')
        amazon_page = soup.select('.a-color-base.a-text-normal')
        amazon_page_length = int(len(amazon_page))
        for i in range(0, amazon_page_length):
            name = name.upper()
            amazon_name = soup.select(
                '.a-color-base.a-text-normal')[i].getText().strip().upper()
            
            # Tính toán độ tương đồng giữa tên sản phẩm và tên sản phẩm tìm kiếm
            similarity_score = fuzz.ratio(name, amazon_name)
            # print(f"Similarity Score: {similarity_score}")
            if similarity_score >= 10:
                amazon_name = soup.select(
                    '.a-color-base.a-text-normal')[i].getText().strip()
                amazon_images = soup.select(
                    '.a-section.aok-relative.s-image-fixed-height')
                amazon_image = amazon_images[0].find_all(
                    'img', class_='s-image')[0]
                amazon_image = amazon_image['src']
                amazon_price = soup.select(
                    '.a-price-whole')[i].getText().strip().upper()
                print("Amazon:")
                print("Tên Sản Phẩm:", amazon_name)
                print("Giá:", amazon_price)
                print("Link Ảnh:", amazon_image)
                print("Link:", amazon_link)
                print("---------------------------------")
                break
            else:
                i += 1
                i = int(i)
                if i == amazon_page_length:
                    amazon_price = '0'
                    print("amazon : No product found!")
                    print("-----------------------------")
                    break

        return amazon_price, amazon_name[0:50], amazon_image, amazon_link
    except:
        print("Amazon: No product found!")
        print("---------------------------------")
        amazon_price = '0'
        amazon_name = '0'
        amazon_link = '0'
        amazon_image = '0'
    return amazon_price, amazon_name[0:50], amazon_image, amazon_link
# ====================================================================================


# *************************Chợ Tốt*************************
def chotot(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "-")
        chotot_url = f'https://www.chotot.com/mua-ban?q={name2}'

        chotot_driver.get(chotot_url) 
        # driver.implicitly_wait(10)  # Chờ đợi element xuất hiện 
        print("---------------------------------")
        print("\nSearching in Chợ Tốt...")
        soup = BeautifulSoup(chotot_driver.page_source, 'lxml')
        
        # Tìm kiếm element
        element_name = soup.select("h3.commonStyle_adTitle__g520j")

        element_price = soup.select("p.AdBody_adPriceNormal___OYFU")
        
        element_image = soup.select("picture.webpimg-container img")


        # Lấy thông tin sản phẩm   
               
        # Giá
        chotot_price = [ele.get_text().strip(' đ').strip().replace(".", "") for ele in element_price]
        chotot_price_float = [float(ele) for ele in chotot_price]
        thresh = np.quantile(chotot_price_float, q = 0.05)
        # remome thresh 
        chotot_price = [ele for ele, price_float  in zip(chotot_price, chotot_price_float) if price_float>=thresh]    
        
        # Name
        # ' '.join(element_names[0].get_text().replace("\n", "").split())
        chotot_name = [' '.join(ele.get_text().replace("\n", "").split()) for ele, price_float in zip(element_name, chotot_price_float) if price_float>=thresh]
                
        # Image
        # element_image[0].attrs['src']
        chotot_image = [ele.attrs['src'] for ele, price_float in zip(element_image, chotot_price_float) if price_float>=thresh]
        
        # Lọc giá bé nhất
        chotot_price_float = [ele for ele in chotot_price_float if ele>=thresh]
        # min(enumerate(a), key=lambda x: x[1])[0]
        index_min = min(enumerate(chotot_price_float), key=lambda x: x[1])[0]
        
        chotot_name = chotot_name[index_min]
        chotot_price = chotot_price[index_min]
        chotot_image = chotot_image[index_min]
        
        print("Chợ Tốt:")
        print("Tên Sản Phẩm:", chotot_name)
        print("Giá:", chotot_price)
        print("Link Ảnh:", chotot_image)
        print("Link:", chotot_url)
        print("---------------------------------")

        return chotot_price, chotot_name[0:50], chotot_image, chotot_url

    except Exception as e:
        print(f"Lỗi: {e}")
        chotot_price = '0'
        chotot_name = '0'
        chotot_image = '0'
        chotot_url = '0'
        return chotot_price, chotot_name, chotot_image, chotot_url
    
# ******************************************


# *************************Sen Đỏ*************************
def sendo(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "-")
        sendo_url = f"https://sendo.vn/tu-khoa/{name2}"

        sendo_driver.get(sendo_url) 
        # driver.implicitly_wait(10)  # Chờ đợi element xuất hiện 
        print("---------------------------------")
        print("\nSearching in Điện Máy Chợ Lớn...")
        soup = BeautifulSoup(sendo_driver.page_source, 'lxml')
        
        # Tìm kiếm element
        element_name = soup.select("span.d7ed-Vp2Ugh._0032-Zwkt7j")

        element_price = soup.select("span._0032-GpBMYp._0032-npoTU_.d7ed-CLUDGW.d7ed-AHa8cD.d7ed-giDKVr")
        
        element_image = soup.select("div.d7ed-a1ulZz img")


        # Lấy thông tin sản phẩm   
               
        # Giá
        sendo_price = [ele.get_text().strip('đ').split("-")[-1].strip().replace(".", "") for ele in element_price]
        sendo_price_float = [float(ele) for ele in sendo_price]
        thresh = np.quantile(sendo_price_float, q = 0.55)
        # remome thresh 
        sendo_price = [ele for ele, price_float  in zip(sendo_price, sendo_price_float) if price_float>=thresh]    
        
        # Name
        # ' '.join(element_names[0].get_text().replace("\n", "").split())
        sendo_name = [' '.join(ele.get_text().replace("\n", "").split()) for ele, price_float in zip(element_name, sendo_price_float) if price_float>=thresh]
                
        # Image
        # element_image[0].attrs['src']
        sendo_image = [ele.attrs['src'] for ele, price_float in zip(element_image, sendo_price_float) if price_float>=thresh]
        # attrs['data-src']
        if not sendo_image:
            sendo_image = [ele.attrs['data-src'] for ele, price_float in zip(element_image, sendo_price_float) if price_float>=thresh]
        
        
        # Lọc giá bé nhất
        sendo_price_float = [ele for ele in sendo_price_float if ele>=thresh]
        # min(enumerate(a), key=lambda x: x[1])[0]
        index_min = min(enumerate(sendo_price_float), key=lambda x: x[1])[0]
        
        sendo_name = sendo_name[index_min]
        sendo_price = sendo_price[index_min]
        sendo_image = sendo_image[index_min]
        
        print("Sen đỏ:")
        print("Tên Sản Phẩm:", sendo_name)
        print("Giá:", sendo_price)
        print("Link Ảnh:", sendo_image)
        print("Link:", sendo_url)
        print("---------------------------------")

        return sendo_price, sendo_name[0:50], sendo_image, sendo_url

    except Exception as e:
        print(f"Lỗi: {e}")
        sendo_price = '0'
        sendo_name = '0'
        sendo_image = '0'
        sendo_url = '0'
        return sendo_price, sendo_name, sendo_image, sendo_url
    
# ******************************************

# *************************Điện máy chợ lớn*************************
def dienmaycholon(name):
    try:
        # Chuẩn bị URL
        name2 = name.replace(" ", "-")
        dienmaycholon_url = f"https://dienmaycholon.vn/tu-khoa/{name2}"

        dienmaycholon_driver.get(dienmaycholon_url) 
        # driver.implicitly_wait(10)  # Chờ đợi element xuất hiện 
        print("---------------------------------")
        print("\nSearching in Điện Máy Chợ Lớn...")
        soup = BeautifulSoup(dienmaycholon_driver.page_source, 'lxml')
        
        # Tìm kiếm element
        element_name = soup.select("h3.name_pro")

        element_price = soup.select("div.price_sale")
        
        element_image = soup.select("a.img_pro img")


        # Lấy thông tin sản phẩm   
               
        # Giá
        dienmaycholon_price = [ele.get_text().strip('đ').split(":")[-1].strip().replace(".", "") for ele in element_price]
        dienmaycholon_price_float = [float(ele) for ele in dienmaycholon_price]
        thresh = np.quantile(dienmaycholon_price_float, q = 0.05)
        # remome thresh 
        dienmaycholon_price = [ele for ele, price_float  in zip(dienmaycholon_price, dienmaycholon_price_float) if price_float>=thresh]    
        
        # Name
        # ' '.join(element_names[0].get_text().replace("\n", "").split())
        dienmaycholon_name = [' '.join(ele.get_text().replace("\n", "").split()) for ele, price_float in zip(element_name, dienmaycholon_price_float) if price_float>=thresh]
                
        # Image
        # element_image[0].attrs['src']
        dienmaycholon_image = ["https:"+ele.attrs['src'] for ele, price_float in zip(element_image, dienmaycholon_price_float) if price_float>=thresh]
        
        # Lọc giá bé nhất
        dienmaycholon_price_float = [ele for ele in dienmaycholon_price_float if ele>=thresh]
        # min(enumerate(a), key=lambda x: x[1])[0]
        index_min = min(enumerate(dienmaycholon_price_float), key=lambda x: x[1])[0]
        
        dienmaycholon_name = dienmaycholon_name[index_min]
        dienmaycholon_price = dienmaycholon_price[index_min]
        dienmaycholon_image = dienmaycholon_image[index_min]
        
        print("Điện Máy Chợ Lớn:")
        print("Tên Sản Phẩm:", dienmaycholon_name)
        print("Giá:", dienmaycholon_price)
        print("Link Ảnh:", dienmaycholon_image)
        print("Link:", dienmaycholon_url)
        print("---------------------------------")

        return dienmaycholon_price, dienmaycholon_name[0:50], dienmaycholon_image, dienmaycholon_url

    except Exception as e:
        print(f"Lỗi: {e}")
        dienmaycholon_price = '0'
        dienmaycholon_name = '0'
        dienmaycholon_image = '0'
        dienmaycholon_url = '0'
        return dienmaycholon_price, dienmaycholon_name, dienmaycholon_image, dienmaycholon_url
    
# ******************************************


C:\Users\truon\AppData\Local\Temp\ipykernel_22920\1945709178.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
d:\anaconda3\envs\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [49]:
dienmayxanh("Macbook pro")

---------------------------------

Searching in Điện Máy Xanh...
Điện Máy Xanh:
Tên Sản Phẩm: Bao da Macbook Pro 13 inch Apple
Giá: 3390000
Link Ảnh: https://cdn.tgdd.vn/Products/Images/1363/313159/bo-dan-macbook-pro-2020-m1-m2-13-inch-6-in-1-innostyle-iscs2338-space-gray-thumb-600x600.jpg
Link: https://www.dienmayxanh.com/tim-kiem?key=Macbook-pro
---------------------------------


('3390000',
 'Bao da Macbook Pro 13 inch Apple',
 'https://cdn.tgdd.vn/Products/Images/1363/313159/bo-dan-macbook-pro-2020-m1-m2-13-inch-6-in-1-innostyle-iscs2338-space-gray-thumb-600x600.jpg',
 'https://www.dienmayxanh.com/tim-kiem?key=Macbook-pro')